Chapter 2.a. Core Expression Types
=======

Expressions in **Prove-It** are fundamental and versatile.  In order to prove some statement to be true, you must be able to express the statement that you want to prove, and express each axiom and each theorem that is used to construct the proof as well as intermediate judgments along the way.  
**Expressions** form the basis for **judgments** that are used in the derivation steps of a **proof** and will be discussed later.
An expression is a tree-like data structure (technically a directed acyclic graph, DAG, since there may be multiple instances of the same sub-expression).  As a Python object, its base class is `proveit._core_.expression.expr.Expression` (also aliased as `proveit.Expression` via "`from ... import ...`" Python statements within `__init__.py` files):

In [1]:
from proveit import Expression
%begin basic_expressions
Expression

proveit._core_.expression.expr.Expression

Each `Expression` object has a class that is derived one of the "primitive" `Expression` classes listed below (each of which is derived from `proveit._core_.expression.expr.Expression`).  These "primitive" types of expressions, in the order that we discuss them below, are:
* <a href="#Variable">`Variable`</a>: A label that is interchangeable (as long as it is kept distinct from different labels) with no intrinsic meaning.  It is often represented by a single letter ($a$, $b$, $x$, $y$, etc.) but can have any representation.
* <a href="#Literal">`Literal`</a>: A label that is not interchangeable and has an intrinsic meaning.  Specific operators ($\lnot, \land, +, \times$, etc.) and specific irreducible values ($\top, \bot, 0, 5$, etc.) are all `Literal`s.  Furthermore, a problem-story `Variable` in a particular theory package, representing some unknown but particular value, should also be a `Literal` (e.g., "Ann has $a$ apples...").
* <a href="#ExprTuple">`ExprTuple`</a>: A list of `Expression`s that is used, for example, when there are multiple *operators* or *operands* of an `Operation`, or multiple *parameters* of a `Lambda`.
* <a href="#Operation">`Operation`</a>: The application of *operator(s)* on *operand(s)*.  For example, $0 + 5 + 8$ and $1 < a \leq b < 3$ are examples of *operation* expressions.  The **Prove-It** library defines many *types* derived from the *operation* type (e.g., for each specific operation), but all *operations* have the same behavior with respect to reduction rules that will be described in a later tutorial.
* <a href="#Conditional">`Conditional`</a>: An expression that has one *condition* and one *value*.  The **Conditional** is reducible to its *value* if and only if the *value* is true.  For example $\{P(x) \textrm{ if } Q(x).$ reduces to $P(x)$ if $Q(x)$ is true.  Otherwise, the **Conditional** is not defined (i.e., it is not reducible).
* <a href="#Lambda">`Lambda`</a>: A mapping defined by *parameter* `Variable`s (and/or *iterations* of *indexed variables* described below with the `Iter` and `IndexedVar` core classes) transforming to some *body* `Expression`.  For example, $(x, y, z) \mapsto \{x+y/z~|~(x \in \mathbb{R}) \land (y \in \mathbb{R}) \land (z \in \mathbb{R}) \land (z \neq 0)\}$ is a `Lambda` with a `Conditional` *body* that converts three real numbers $x, y, z$ to $x + y/z$ as long as $z$ is not zero.  Note that a `Lambda` introduces `Variables` into a new scope via the *parameters*.  These *parameter* `Variable`s are said to be *bound* in this new scope; occurrences outside this scope are not deemed to be the same thing. There is a special category of `Operation` types called `OperationOverInstances` that are essentially functionals acting on a `Lambda` operand.  $\forall$, $\exists$, $\sum$, and $\prod$ are examples of `OperationOverInstances`.  For example, $\forall_{x~|~Q(x)} P(x)$ is an expression that translates to "$P(x)$ is true for all values of $x$ for which $Q(x)$ is also true".  Internally, this is represented as an $\forall$ operator acting on the `Lambda` map with a `Conditional` *body*: $x \mapsto \{P(x)~|~Q(x)\}$.
* <a href="#NamedExprs">`NamedExprs`</a>: A mapping from keyword strings to `Expression`s.  This can be used to prevent ambiguity of an expression's internal representation.
* <a href="#ExprRange">`ExprRange`</a>: Represents a range of expressions with a *parameter* going from a *start* index to an *end* index in successive unit increments ($+1$).  It contains a `Lambda` to define each expression in the range as a function of the *parameter* value (the index).
For example, $1/(x+i) + ... + 1/(x+j)$ is essentially an `Operation` with `+` as the *operator* and an `ExprRange` as the *operand*.  This `ExprRange` in this case is $1/(x+i), ..., 1/(x+j)$.  The sub-expression of this `ExprRange` may be denoted as $k \mapsto \{1/(x+k)~|~k \in i .. j\}$ where $k$ is an arbitrary `Lambda` *parameter* and $i .. j$ denotes an integer interval. 
* <a href="#IndexedVar">`IndexedVar`</a>: A special kind of `Operation` that indexes a `Variable`.  It treats the `Variable` being indexed as the *operator* and the *index* as the *operand*.  For example: $a_1$, $x_i$, $x_{i+1}$, etc.  An *iteration* if *indexed* *variables* acts as a collection of labels, however.  For example: $a_1, ..., a_n$ is a collection of $n$ labels and can be used as *parameters* of a `Lambda` map and has special substitution rules in which an `Iter` may be expanded to multiple entries within an `ExprTuple`.

These "primitive" types are special for the following reason.  In order to automatically or manually verify the correctness of a proof in **Prove-It**, one is only required to know the primitive base class of the expressions in the expression DAGs involved in the proof.  The way that the expression is to be formatted may be defined by the derived class, but this is a dressing which is not actually verified by **Prove-It** in any way.  For this reason, in order to really understand the axioms that were used in a proof, or understand what was actually proven, one must view the full expression DAG and understand its structure with respect to the primitive types.

Other classes are derived from these primitives for the purpose of defining specific ways of formatting the expression (as a string or as $\LaTeX$) and to provide methods for convenience and automation for manipulating expressions of that specific type.  The most common "primitive" type used as a base class is the `Operation`; there are a wide variety of kinds of operations (logical operations such as $\land$, $\lor$, $\lnot$, etc.; the $\forall$ and $\exists$ quantifiers, number operations such as $+$, $-$, etc.).  Each one of these has its own methods that are convenient for applying axioms and theorems specific to the operation.

A few non-privitive expression casses are define in the core (`proveit._core_`) for organizational purposes.  These are:
* <a href="#Function">`Function`</a>: An `Operation` that uses function-style formatting by default.  For example, $f(x)$, $P(x, y)$, etc.  Beyond formatting, these are no different than `Operation` types.
* <a href="#OperationOverInstances">`OperationOverInstances`</a>: An `Operation` whose operand is a `Lambda` expression.  Often, these `Lambda` expression has a `Conditional` body.  The idea is that it operates over the domain of instances of the `Lambda` parameter variables for the the *condition* of the `Conditional` is satisfied.  Quantifiers are `OperationOverInstances`.  $\forall_{x, y~|~Q(x, y)} P(x, y)$ is internally represented by Prove-It as an operation of the $\forall$ operator acting on $(x, y) \mapsto \left\{P(x, y) \textrm{ if } Q(x, y)\right.$.  $\sum_{k = 1}^{N} f(x)$ is internally represented by Prove-It as an operation of the $\sum$ operator acting on $k \mapsto \left\{f(x) \textrm{ if } k \in 1 .. N\right.$.
* <a href="#ConditionalSet">`ConditionalSet`</a>: An expression that has one or more *conditions* with associated *values*.  For example, consider this definition for the absolute value of an integer: $\{x~\textrm{ if }~x \geq 0; -x~\textrm{ if }~x < 0.$ A *value* is only pertinent if its associated *condition* is satisfied (a **judgment**).  Via axioms, a `Conditional` equates to one of its *values* if its associated *condition* is proven the be true and all of its other *conditions* are proven to false.  With respect to substitution rules, the substitution of a *value* may utilize the fact that its *condition* must be satisfied in order to be pertinent, and therefore its *condition* may be assumed within the theory of performing substution for the *value*.

Developers and users may derive new classes from core `Expression` classes (`Literal` and `Operation` classes in order to make their own `Expression` classes that have special formatting rules for displaying the **expression** and methods for manipulating and utilizing the expression (i.e., applying theorems or axioms as will be discussed in later tutorial chapters).  Ideally, the formatting should be in direct correspondence with the structure of the `Expression` as a true reflection of the internal representation, but there is no enforcement mechanism to ensure that this is the case.  Users are advised to inspect the fully explicit expression DAGs of important axioms and theorems (or rely on crowdsource checking).  Conveniently, clicking on any **expression** rendered as LaTeX will link to a notebook that, when executed, will reveal this internal structure.  (Alternatively, the `expr_info()` method may be called).

The remainder of this tutorial chapter will show examples and discuss details of the different core `Expression` classes and discuss their distinct behaviors with respect to *substitution* as well as *meaning* versus *style* equivalence.

The following are related but distinct manipulations in **Prove-It**:
* *Expression substutition*: creating a new **expression** from an existing **expression** by swapping one or more sub-expression(s) for other sub-expression(s) by using the `substituted` method.  This creates a new `Expression` but has nothing to do, intrinsically, with proving statements.
* *Instantiation* derivation step: deriving a new **judgment** from an existing **judgment** via *expression substitution* using specific rules and limitations to ensure that the derivation is sound.  This will be discussed in a later tutorial chapter.
* *Equality substitution*: using the `substitution` axiom of `proveit.logic.equality` to prove that $f(x) = f(y)$ given $x = y$.  This uses *instantiation* of the `substitution` axiom which states $\forall_{f, x, y~|~x=y}~(f(x) = f(y))$.  This specifically involves statements (**judgments**) with the `proveit.logic.equality.equals._operator_` **Literal** (the `=` sign). This will be discussed in a later tutorial chapter.

The focus here will be *expression substitution* with some mention of the additional restrictions applicable to the *instantiation* derivation step before delving into more details in later chapters.  It is important to understand the distinctions among these types of manipulations.

<a name="#Label"></a>Labels (Variables, IndexedVars, and Literals)
================================

In [2]:
from proveit import Label

The `Variable` and `Literal` classes both derive from `Label`.  A `Label` is created with a string and a LaTeX format specification to determine how it is displayed.  In a Jupyter notebook, the LaTeX is rendered and presented in the output:

In [3]:
tri = Label(string_format='triangle', latex_format=r'\triangle')

tri: triangle

After importing anything from `proveit` in a Jupyter notebook, whenever assignments are made at the end of an input cell the output shows the name of the assigned Python variable (not to be confused with a **Prove-It** *Variable*) followed by ':' and its rendered value.  In the above input, we have assigned `tri` to a `Label` object we have created which renders as a triangle shape.  For convenience, this Python variable name and shape appear in the output without any additional code.  The string format is presented when the object is converted to a string and is a useful alternate representation (used, for example, in error messages that are displayed as strings).

In [4]:
str(tri)

'triangle'

The breakdown of an **expression** into its DAG structure is displayed by calling `expr_info()`.  Clicking on the **expression** is also a way to view this information, linking to a page that shows a canonical way to build the **expression** and calling `expr_info()` on that built **expression**.  Here we see that the `tri` is a `Label` with no sub-expressions.

In [5]:
tri.expr_info() # the expressions and sub-expressions are numbered with the top level being zero.

,core type,sub-expressions,expression
0,Label,,


`Label` was not one of the core **expression** types listed above because it is not intended to be used except as the base class of `Variable`, `IndexedVar`, and `Literal`.  Notwithstanding, `Label` is displayed as the core type.
Two Label's are regarded to be the same expression if and only if the class and both formats (string and LaTeX), are the same:

In [6]:
assert tri == Label('triangle', r'\triangle') # equal b/c class (Label) is the same and
                                              # the string and LaTex formats are the same
    
assert tri != Label('tri', r'\triangle')      # not equal when class or either format is different
                                              # Class for both is Label, but string format different

Let's make another label to test out `substituted()` method.

In [7]:
sq =  Label(string_format='square', latex_format=r'\Box')

sq: square

Using *expression substitution*, we can change one `Label` into another or into any other `Expression`.  Later we'll see that *instantiation* is more restrictive, only being applicable to `Variable`s and not any `Label`, but for *expression substitution* this is fine.

In [8]:
tri.replaced({tri:sq})

square

Notice that the `replaced()` method is non-mutating, leaving the tri `Label` as it was originally defined:

In [9]:
tri

triangle

<a href="#Variable"></a>Variable
===========
A `Variable` is an interchangeable <a href="#Label">Label</a> with no formal, contextual meaning.

In [10]:
from proveit import Variable
x = Variable('x')  # the string and latex formats
                   # are the same by default

x: x

In [11]:
# different string and latex formats:
omega = Variable(string_format='omega', latex_format=r'\omega') 

omega: omega

Convenient `Variable`s and other `Expression`s are accessible from the main `proveit` theory package
as defined in <a class="ProveItLink" href="../packages/proveit/_theory_nbs_/common.ipynb">proveit.\_theory\_nbs\_/common.ipynb</a>.

In [12]:
from proveit import a, b, c, x, y, z, alpha

In [13]:
from proveit import ExprTuple
ExprTuple(a, b, c, x, y, z, alpha)

(a, b, c, x, y, z, alpha)

We used an <a href="#ExprTuple">ExprTuple</a> to conveniently render all of those `Variable`s as a single `Expression`.

Expression information for a `Variable` is similar to that for a `Label`, but with the different core type and class:

In [14]:
alpha.expr_info()

,core type,sub-expressions,expression
0,Variable,,


Just as we saw for the generic `Label` `Expression`s, we can use *expression substitution* to exchange one `Variable` for another or for any other `Expression`.  Furthermore, we will see in a later tutorial chapter that `Variable`s have special properties with respect to *instantiation* because they are defined as interchangeable labels.

In [15]:
alpha.replaced({alpha:sq}) # we can substitute a Variable for any other Expression

square

<a name="#Literal"></a>Literal
-------

A `Literal` is another kind of <a href="#Label">Label</a>.  In contrast to `Variable`s, `Literal`s have a formal, contextual meaning.

In [16]:
from proveit import Literal

Below are different scenarios in which a `Literal` could appropriately be used.

### Specific operators

In [17]:
TIMES = Literal(string_format='*', latex_format=r'\times')

Expression notebook is being updated for *


TIMES: *

In [18]:
FACTORIAL = Literal('!')

Expression notebook is being updated for !


FACTORIAL: !

In [19]:
SUMMATION = Literal('sum', r'\sum')

SUMMATION: sum

We will see in the <a href="#Operation">Operation</a> section that the *operator* of an `Operation`-derived class should be stored as a Python class variable called `_operator_`.  We are using module-level Python variable names just for these examples.

### Constant values

In [20]:
FIVE = Literal('5') # Literal constants are also ALL-CAPS

FIVE: 5

In [21]:
TRUE = Literal('true', r'\top')

TRUE: true

In [22]:
FALSE = Literal('false', r'\bot')

FALSE: false

Such constants will typically be defined as **common expressions** in the appropriate **theory**, to be discussed in a later tutorial.

### Theoryual "variables"

This is appropriate when variables are given a specific meaning within the theory of a problem.  For example, consider a math "story problem" where Andrea has $a$ apples and Bill has $b$ bananas.  These are variables in a sense, but here they have contextual meaning so they should be `Literal`s.  However, suppose one wants to prove a general theorem for any number of Andrea's apples and Bill's bananas.  One may start by using the $a$ and $b$ `Literal`s for convenience for some contextual "lemmas" (as an aside, though, **Prove-It** does not distinguish between lemmas and theorems), but then demote them to `Variable`s in order to make a statement of the form $\forall_{a, b} P(a, b)$ (`Literal`s may not be quantified over).  This is done by using a process called axiom elimination that will be discussed later.  For now, we just note that "variables" with contextual meaning need to be `Literal`s.

In [23]:
a_lit = Literal('a')

a_lit: a

In [24]:
b_lit = Literal('b')

b_lit: b

In [25]:
# different because one is a Variable and one is a Literal
assert a != a_lit 
# same after converting the Literal to a Variable
assert a == a_lit.as_variable() 

### Literal substitution
It is possible to perform *expression substitution* on a `Literal`.

In [26]:
a_lit.replaced({a_lit:b_lit})

b

In [27]:
a_lit.replaced({a_lit:omega})

omega

As noted earlier, the `substituted()` method is non-mutating, leaving the a_lit `Literal` as it was originally defined:

In [28]:
a_lit

a

While we are able to replace a `Literal` via *expression substitution*, we may not replace them via *instantiation* because `Literal`s are not interchangeable labels.

### Literal theories

A `Literal` is distinguished not only by its formatting but also by its **theory**.  Axioms and theorems of **Prove-It** are organized via **theories** which we will discuss later in more detail.  Each `Literal` belongs to a **theory** which is typically the one corresponding to the package (directory) in which it is defined.  This is particularly important for *contextual variable* types of `Literal`s that are very *theory*-specific.

In [29]:
from proveit import logic

In [30]:
logic.TRUE

TRUE

*If the above output is not the $\top$ symbol when you execute this, you probably need to <a class="ProveItLink" href="tutorial00_introduction.ipynb#build">build or download the Prove-It database</a>.*

In [31]:
# When the theory is different, they are not the same.
TRUE == logic.TRUE

False

The detailed expression information reveals the differences:

In [32]:
TRUE.expr_info(details=True)

0. true
   core type: Literal
   latex_format: \top
   theory: tutorial
   class: <class 'proveit._core_.expression.label.literal.Literal'>
   sub-expressions:

In [33]:
logic.TRUE.expr_info(details=True)

0. TRUE
   core type: Literal
   latex_format: \top
   theory: proveit.logic.booleans
   class: <class 'proveit.logic.booleans.booleans.TrueLiteral'>
   sub-expressions:

**Expressions** are also distinguished by their class (derived from the *core type*).  In the above example, `logic.TRUE` is an object of the `proveit.logic.booleans.booleans.TrueLiteral` class.  `logic.TRUE` and `logic.FALSE` have an `eval_equality(..)` method defined in their respective classes which is a convenient method for deriving any of the following: $(\top = \top) = \top$, $(\bot = \bot) = \top$, $(\top = \bot) = \bot$, or $(\bot = \top) = \bot$ (these are proven theorems within `proveit.logic`). For example:

In [34]:
logic.TRUE.eval_equality(logic.FALSE)

|- (TRUE = FALSE) = FALSE

That is a sneak preview of a judgment (**Judgment** object) that uses the turnstile notation, $\boldsymbol{\vdash}$, to indicate that the expression is a proven statement.  There will be more on this in later tutorial chapters.

<a name="ExprTuple"></a>ExprTuple
=========

An `ExprTuple` is an ordered collection of **expression** objects which we previewed briefly already.  Here is another example:

In [35]:
et = ExprTuple(a, b, c, TRUE, FIVE, a, alpha)

et: (a, b, c, true, 5, a, alpha)

An `ExprTuple` may be used as the *operands* or *operations* of an `Operation`, the *parameters* of a `Lambda` map, or the *values* and *conditions* of a `Conditional`.  A Python tuple (or any iterable) will automatically be converted to an `ExprTuple` when it is passed in for any of these roles when constructing an `Operation`, `Lambda`, or `Conditional`. 

Substitution typically carries forward in a straightforward manner.

In [36]:
et.replaced({a:FALSE})

(false, b, c, true, 5, false, alpha)

Note that the *substitution* occurs on all relevant instances.  And, as always, `substituted` is non-mutating:

In [37]:
et

(a, b, c, true, 5, a, alpha)

The **expression** constituents of an `ExprTuple` are known as *entries*:

In [38]:
et.entries[5]

a

The enumerated mathematical objects represented by the `ExprTuple` are known as *elements*.  The *elements* and *entries* are in correspondence except when the `ExprTuple` contains an `Iter`.  A contained `Iter` is a single *entry* but may represent multiple elements.  For example, the second *entry* of $(a, b_1, ..., b_n)$ is the `Iter` $b_1, ..., b_n$, but the second *element*, assuming $n \geq 1$, is $b_1$.  We will revisit this in the <a href="#Iter">Iter</a> section.

`ExprTuple` *substitution* is more interesting when an `Iter` containing an `IndexedVar` is expanded such that the `Iter` splits into multiple *entries*.  This will be demonstrated in the <a href="#IndexedVar">IndexedVar</a> section.  

<a name="Operation"></a>Operation
=========

This type of **expression** represents an applied operation.  It contains *operator(s)* and *operand(s)*.  The default formatting of an `Operation` with one *operator* and more than one *operand* is to place the *operator* between each successive pair of *operands* as follows.

In [39]:
from proveit import Operation
triple_prod = Operation(TIMES, [x, y, omega])

Must implement 'extract_init_arg_value' for the Operation of type <class 'proveit._core_.expression.operation.operation.Operation'> if it does not fall into one of the default cases for 'extract_init_args'


triple_prod: x * y * omega

A `Function` is derived from `Operation`.  It behaves in the same manner except it formats it in a "function" style:

In [40]:
from proveit import Function

# Simple case: a Variable operator with a single Variable operand
f = Variable('f')
fx = Function(f, x)

fx: f(x)

In [41]:
from proveit import fx, gx, Px, Qx
ExprTuple(fx, gx, Px, Qx)

(f(x), g(x), P(x), Q(x))

An `Operation` must have a single *operator* that is a *label* (*variable* or *literal*).

In [42]:
try:
    Operation(fx, fx)
    assert False, "Expecting a TypeError; should not make it to this point"
except TypeError as e:
    print("Expected TypeError: %s"%str(e))

Expected TypeError: operator must be a Label or an indexed variable (IndexedVar). f(x) is none of those.


The `operator` and `operands` attributes access the *operator* and *operands* respectively as `ExprTuple`s:

In [43]:
triple_prod.operator

*

In [44]:
triple_prod.operands

(x, y, omega)

If there is only one *operand*, it may be accessed via the `operand` attributes: 

In [45]:
fx.operand

x

In [46]:
# An Operation does not have an `operand` attribute
# if there is more than one *operand*.
hasattr(triple_prod, 'operand')

False

Looking at the expression information, we see that `triple_prod` has `operands` that is an `ExprTuple` but a single `operator` that is a `Literal`.

In [47]:
triple_prod.expr_info()

0. x * y * omega
   core type: Operation
   operator: 1
   operands: 2
1. *
   core type: Literal
   sub-expressions: 
2. (x, y, omega)
   core type: ExprTuple
   sub-expressions: 3, 4, 5
3. x
   core type: Variable
   sub-expressions: 
4. y
   core type: Variable
   sub-expressions: 
5. omega
   core type: Variable
   sub-expressions: 

### Defining an `Operation` sub-class

Let's derive a couple of classes from `Operation` to represent operations with specific literal operators.
This is commonly done within Prove-It library modules.  We use the `%load` magic command to display (and execute) the contents of <a href="demo_operations.py">demo_operations.py</a>.

In [48]:
# %load demo_operations.py
'''
Module that defines a Operations for demonstration
purposes in this tutorial.
'''

from proveit import Operation, Literal

class Factorial(Operation):
    # _operator_ is a special class variable name, defining specific literal operator of the Operation class.
    # It is not only used for default formatting but also when performing substitutions for rebuilding expressions.
    _operator_ = Literal('!')
    
    def __init__(self, operand):
        # creates the Operation with FACTORIAL as the operator and the provided operand as its only operand.
        Operation.__init__(self, Factorial._operator_, operand) # initializes self.operand

    def string(self, **kwargs): # should accept kwargs even when not used (e.g., 'fence')
        # the operand should be fenced (wrapped in parentheses) to prevent ambiguity
        return self.operand.string(fence=True) + Factorial._operator_.string()
    
    def latex(self, **kwargs): # should accept kwargs even when not used (e.g., 'fence')
        # the operand should be fenced (wrapped in parentheses) to prevent ambiguity
        return self.operand.latex(fence=True) + Factorial._operator_.latex() 

class Multiply(Operation):
    
    # This operator Literal has a LaTeX format that differs from the string format.
    _operator_ = Literal('*', r'\times')
    
    def __init__(self, *operands): # takes a list of arguments as the operands
        # creates the AssociativeOperation with TIMES as the operator and any number of operands.
        Operation.__init__(self, Multiply._operator_, operands)
    
    # The default formatting will display the operator between the operands


In order to work properly, however, we need to import these classes from the module (as Prove-It will use the `__file__` attribute of this module for its internal purposes).

In [49]:
from demo_operations import Factorial, Multiply

A `Factorial` is an `Operation` with "!" as the format of the `Literal` operator.  Its formatting is altered from the default but is still true to the core `Expression` structure (the formatting is a reflection of the internal structure).

In [50]:
# Now we can make an object with this new class
x_factorial = Factorial(x)

x_factorial: x!

In [51]:
# show core structure
x_factorial.expr_info()

Expression notebook is being updated for !


,core type,sub-expressions,expression
0,Operation,operator: 1operand: 3,
1,Literal,,
2,ExprTuple,3,
3,Variable,,


In the Jupyter notebook setting, we typically only use LaTeX formatting, but it is important to define the string formatting as well for whenever it is needed (e.g., when error messages are displayed).  The `fence=True` above indicates that parentheses should be used when it could be ambiguous otherwise.  Our example above has no ambiguity, so parentheses are not used.  We will show a case below where the parentheses are required.

Next we consider a multiple *operand* example using the `Multiply` class that we imported from <a href="demo_operations.py">demo_operations.py</a> above.

In [52]:
# Demonstrating an Operation with multiple operands
mult_expr = Multiply(x, FIVE, omega)

mult_expr: x * 5 * omega

In [53]:
# Let's nest Operations and show proper fencing behavior
nested_operation = Factorial(mult_expr)

nested_operation: (x * 5 * omega)!

In [54]:
nested_operation.expr_info()

Expression notebook is being updated for *


,core type,sub-expressions,expression
0,Operation,operator: 1operand: 3,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAGkAAAAUBAMAAAB8EA4WAAAAMFBMVEX////MzMxUVFR2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAACgK8IjAAAAAXRSTlMAQObYZgAAAYtJREFU OMuVlL9Lw0AUx79tWtv8aMzaRbp1UoqDRRDMppMI6ij6B4h2UgodUhARp4i4ORw4W3D2V0EKgoMd FF0sGatLI4pFEPXummpSLkIf3B33ee97vHv3EkCvow9b9dZrofeoml4W8agDJDNARaha+n4Vco0A rVmkbKH3MOuIU8wDNxaSYm8m7GI1YJCg6UfHdDwHVH/EM8VEPIMiS3ZjZCrPUNOAXubO4ftdBEiU Dn0TkGwoNs7o7hIPhR3mogE0jNkpWmaA3LLT20CKQC1gi+4mMFmd6Rw9VO6moloBwos2BgxYkOuY p5sqTrxQfdvoquQvP4mZUIAGEHEhmVwFtHtUUQLpzU8SQBrIcVXK4Bki8uKpuvnECeRPP5GBdehU FbOggVdjJeGiFLg7DUoSP1GNyAcUh1eD2jiVv6tup4d/66zbuHP8RJrbX9tbZOflUCOYpgXNloqP PW/6NHoQINpFJbFQYK9ssO4N6ahQu+JzSPeGWr6znPcl0sh/X2WYqd6F+vsDNPj8A8P5ahVoYdoU AAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
1,Literal,,
2,ExprTuple,3,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAGMAAAAUBAMAAABrMp7fAAAAMFBMVEX////MzMxUVFR2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAACgK8IjAAAAAXRSTlMAQObYZgAAAWdJREFU KM+Vk7FLw0AUxr82sWmSNmbtom5OQnBQ3Dq6Oaij2L9Ai4LiZApSHSPiphBwtuiuNghSwaWD4iYZ HKIgCgoiFPVdbDBX7oY+uDvu9+477rt7B1ht9BFLrLsRpo6DUkXEsyF1DaFk8eddyAs+UPSEqaPR UHyySSAvTo3IzFwBUXp+Su2Zk/yTbhhlbNDQWnlcrbJ5ZMOqxZmx+11wJEvN2gIUD+dkySlu533G KUtrWJzhtcyRW+b9k7z7qJMwUDrqn6VoqJacwHQ5Et/SBJBzMUdimCfdddaOnUj07zRRyzCAByDz xiTAYLVHkvWhfKSJBpQAJ5bUGRhO9k6OMeBD76SJDqzBIonqMvu5YBNawJmlFfyFmHbmC0YY258C ZipNvPBXanm4C9NEmd1f3ltgmzmYBi7XW4ftnod7Gj/gSKHZ0OaZY8OWFYw0rqVlKQ0qS1z0pWDF L/lisjCZj/4+Mr3/LxlWY6NoCYKGAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
3,Operation,operator: 4operands: 5,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAFYAAAAOBAMAAABHpfZeAAAAMFBMVEX///8QEBBmZmaqqqrMzMxE REQyMjIiIiKYmJh2dna6urrc3NxUVFSIiIju7u4AAAA6P00QAAAAAXRSTlMAQObYZgAAAPVJREFU KM9jYMACMh44GTAQCeT//yRWKUPm2Q1Eqz1AUEUIEG9DUYsQgYK3d3bdvABibHdg4C4BC53oVWFA EeEBYu4qBr4JHLXsCSABoDBQEgTiGPYvQBHpAmK+vwycDzj/cUG8st27BGYZawGKiAKImMzAx8Aa AFXAXe4AU8v0G1mEawEDGwNDN5DFfwFNLU8CA+cvZBFeBgYnBoYJQJY/zDSYjYwJDEz/kEWYGBgu M3BPYGB+cJ+B9wGKT4BSqL5ldWD5xMC2gWG9QTzDDtQQ4lZgaNyALMIpqWivKcHA8ObuW40GtJDf NEcPRYQvPptX8AIDAOdZR7o2Dx8AAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
4,Literal,,
5,ExprTuple,"6, 7, 8","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEMAAAAUBAMAAAAkb50PAAAAMFBMVEX////MzMxUVFR2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAACgK8IjAAAAAXRSTlMAQObYZgAAAVZJREFU KM9jYOC7wIAH5IGI0yhCaw9IJiDzmR4AiXUoSuL/f0bh8yxgYOCdgCK0WP0Bqk3mDAwcqEIK6I45 ysDwggG/Ei4HhkqQheV67uYQEd3rE6HuBGK+FgYG5gkMe4HMwww3CrogMnsY3juAGVdAWr8D3bqA oQ3ItGGwP+AHM5m7AUyBfWHGwMDWwBACZB1g2I2wnPMfiGRxYOBiYLjHwMD4AayEgeE7LKQWMDB/ ATHYGRgkGRgMwEpAFjEwfoQqYV3AwPkHbBYDQxkDH1AJSwPYuTnsHxhqGKAyHAvALhJg/MnA9QDs XEugwm/cH4CReR4oxzeB4eoDnl9AvwZPzZ8cC9JkwOAB9Jt6TeVdoPYeoMhLozkMDB0HGHj2r2MP LQAFnQByBOTAGGwHEB48gRKNBjAGM2o0MuyDx3sBjHUcNTEgkhQvXBwpHXKD3IE/YQLDFwD+aE8i k6qGQAAAAABJRU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" />"
6,Variable,,
7,Literal,,
8,Variable,,


### Operand substitution

*Substitution* of *operands* is straightforward in the way it works.

In [55]:
x_factorial.replaced({x:omega})

omega!

Notice that the substituted() method is non-mutating, leaving the **x_factorial** expression as it was originally defined:

In [56]:
x_factorial

x!

We can also substitute an entire expression in for the variable $x$:

In [57]:
nested_operation_from_sub = x_factorial.replaced({x:mult_expr})

nested_operation_from_sub: (x * 5 * omega)!

Internally, however, there is specific machinery required to regenerate these objects and construct them with the proper classes.  In order for this machinery to work, the `_operator_` class attribute must be properly defined (e.g., `Factorial._operator_` and `Multiply._operator_`).  To demonstrate that this is working properly, note that the classes displayed in the detailed expression info from `expr_info()` are as they should be:

In [58]:
nested_operation_from_sub.expr_info(details=True)

0. (x * 5 * omega)!
   core type: Operation
   class: <class 'demo_operations.Factorial'>
   operator: 1
   operand: 3
1. !
   core type: Literal
   latex_format: !
   theory: tutorial
   class: <class 'proveit._core_.expression.label.literal.Literal'>
   sub-expressions: 
2. (x * 5 * omega)
   core type: ExprTuple
   class: <class 'proveit._core_.expression.composite.expr_tuple.ExprTuple'>
   sub-expressions: 3
3. x * 5 * omega
   core type: Operation
   class: <class 'demo_operations.Multiply'>
   operator: 4
   operands: 5
4. *
   core type: Literal
   latex_format: \times
   theory: tutorial
   class: <class 'proveit._core_.expression.label.literal.Literal'>
   sub-expressions: 
5. (x, 5, omega)
   core type: ExprTuple
   class: <class 'proveit._core_.expression.composite.expr_tuple.ExprTuple'>
   sub-expressions: 6, 7, 8
6. x
   core type: Variable
   latex_format: x
   class: <class 'proveit._core_.expression.label.var.Variable'>
   sub-expressions: 
7. 5
   core type: Literal
   latex_format: 5
   theory: tutorial
   class: <class 'proveit._core_.expression.label.literal.Literal'>
   sub-expressions: 
8. omega
   core type: Variable
   latex_format: \omega
   class: <class 'proveit._core_.expression.label.var.Variable'>
   sub-expressions:

Furthermore, this expression that we obtained via substitution is equivalent to the one constructed directly:

In [59]:
print("Expressions, generated in different ways, are the same:", 
      (nested_operation_from_sub == nested_operation))

Expressions, generated in different ways, are the same: True


### Operator substitution

The *operator* itself may likewise be *substituted*, but with some interesting extra capabilities.

It is straightforward to substitute a **Variable** *operator* with another **Variable** (i.e., relabel). Here we relabel the $f$ in $f(x)$ to $g$ to obtain $g(x)$:

In [60]:
from proveit import g
fx.replaced({f:g}) # Variable operator to a different Variable

g(x)

And it is straightforward to *substitute* a **Variable** *operator* with a **Literal**.

In [61]:
x_factorial_from_fx = fx.replaced({f:FACTORIAL})

x_factorial_from_fx: x!

In [62]:
x_factorial_from_fx.expr_info(details=True)

0. x!
   core type: Operation
   class: <class 'demo_operations.Factorial'>
   operator: 1
   operand: 3
1. !
   core type: Literal
   latex_format: !
   theory: tutorial
   class: <class 'proveit._core_.expression.label.literal.Literal'>
   sub-expressions: 
2. (x)
   core type: ExprTuple
   class: <class 'proveit._core_.expression.composite.expr_tuple.ExprTuple'>
   sub-expressions: 3
3. x
   core type: Variable
   latex_format: x
   class: <class 'proveit._core_.expression.label.var.Variable'>
   sub-expressions:

In [63]:
assert x_factorial_from_fx == x_factorial # Same

It is able to generate the new **expression** in the appropriate `Factorial` class via the internal mechanisms that use its `_operator_` class attribute.

Performing an *expression substitution* of a `Literal` *operator* within an `Operation` class will typically be blocked because it will not know how to make the new **expression**:

In [64]:
from proveit import OperationError
# But this will typically be prevented (appropriately) in trying to remake a derived Operation class:
try:
    x_factorial.replaced({FACTORIAL:SUMMATION})
    assert False, "Expecting an OperationError error; should not make it to this point"
except OperationError as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: An implicit operator may not be changed


### <a name="Operation_sub"></a>Operation substitution

The more interesting and useful case is to *substitute* the `Operation` itself. Here we convert $f(x)$ to $x!$ by substituting a `Lambda` mapping for the function variabe $f$:

In [65]:
from proveit import Lambda
fancy_factorial_map = Lambda(y, Multiply(y, Factorial(y)))

fancy_factorial_map: y -> (y * y!)

The <a href="#Lambda">`Lambda`</a> core type will be discussed in more detail below.  We use it here for a quick example.

In [66]:
fx.replaced({f:fancy_factorial_map})

x * x!

Note that the operand is still $x$ (not $y$).  This is substituting the *operation*, not the operand.  If desired, however, it could be substituted simultaneously:

In [67]:
fancy_omega_fn = fx.replaced({f:fancy_factorial_map, x:omega})

fancy_omega_fn: omega * omega!

In [68]:
fancy_omega_fn.expr_info(details=True)

0. omega * omega!
   core type: Operation
   class: <class 'demo_operations.Multiply'>
   operator: 1
   operands: 2
1. *
   core type: Literal
   latex_format: \times
   theory: tutorial
   class: <class 'proveit._core_.expression.label.literal.Literal'>
   sub-expressions: 
2. (omega, omega!)
   core type: ExprTuple
   class: <class 'proveit._core_.expression.composite.expr_tuple.ExprTuple'>
   sub-expressions: 6, 3
3. omega!
   core type: Operation
   class: <class 'demo_operations.Factorial'>
   operator: 4
   operand: 6
4. !
   core type: Literal
   latex_format: !
   theory: tutorial
   class: <class 'proveit._core_.expression.label.literal.Literal'>
   sub-expressions: 
5. (omega)
   core type: ExprTuple
   class: <class 'proveit._core_.expression.composite.expr_tuple.ExprTuple'>
   sub-expressions: 6
6. omega
   core type: Variable
   latex_format: \omega
   class: <class 'proveit._core_.expression.label.var.Variable'>
   sub-expressions:

Note that by substituting $f$ with a `Lambda` expression, the entire `Operation` is substituted, not just the *operator*.  This is why `Lambda` *operator*s are not allowed (only `Label` operators); otherwise, such a substitution could be ambiguous or confusing (should it substitute the `Operation` or substitute the *operator* with the `Lambda` expression?).  This also highlights the fact that **Prove-It** is not a functional programming or lambda calculus.  Rather, **Prove-It** is designed for manipulating expressions as desired, and *operation substitution* is a useful and powerful, yet relatively straightforward tool for doing so.

<a name="Conditional"></a>Conditional
====

A `Conditional` has one or more *values* associated with one or more *conditions*.  A given *value* is only pertinent if its associated *condition* is satisfied (known to be true).  MUST UPDATE THIS SECTION.  A CONDITIONAL ONLY HAS ONE VALUE AND CONDITION.  A CONDITIONAL SET

In [69]:
from proveit import Conditional, ConditionalSet

To create the `Conditional`, supply the *values* and *conditions* as separate arguments in the constructor.  This example invokes the `Neg`, `Less`, and `greater_eq`  <a href="#Operation">`Operation`</a>s as well as the `zero` <a href="#Literal">`Literal`</a>)

In [70]:
from proveit.numbers import zero, Neg, Less, greater_eq
abs_cond = Conditional((x, Neg(x)), 
                       (greater_eq(x, zero), Less(x, zero)))

abs_cond: {(x, -x) if x >= 0 ,  x < 0.

In [71]:
# The string formatting is somewhat different:
str(abs_cond)

'{(x, -x) if x >= 0 ,  x < 0.'

The `conditions` and `values` attributes may be used to access the *conditions* and *values* respectively.

In [72]:
abs_cond.condition

(x >= 0) and (x < 0)

In [73]:
abs_cond.value

(x, -x)

This is the expression information breakdown for our `Conditional` example.

In [74]:
abs_cond.expr_info()

,core type,sub-expressions,expression
0,Conditional,value: 1condition: 2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAN8AAAAVBAMAAAA0pCbNAAAAMFBMVEX///+IiIi6urrc3NxERETM zMwQEBCqqqpUVFQiIiJ2dnYyMjLu7u5mZmaYmJgAAABN0h6tAAAAAXRSTlMAQObYZgAAAolJREFU SMdjYGDgu8dAV8CzC0Ta4ZRnN6CyhXwfQLY64FbQDGOYPWBg0BDBpWzhqwA8tiDJck0AEtx4vHEa xohfwMCao6+AJHXyDpzJXcAzGbcZyLJgC9/icRyXAILNixYS1XDvP2NgWI01Rraiy4ItnIwvkhMQ bI4LaJLarVCGBwND/QEsETbXAF0WZCHzBHyxvAlKmwZgWsgQ4gShfzAw6BdgaGWcdQBDFmQhI9Ac HsFmxYWoyg9KGEkBw3MthMfs38B6LfcCkgQYWGwHB8NfoJEQISSDTk2EBhJCFhhKGxgYWIDp7xTD S4HZqGG5gG8iJ1D2NSzmGxg4GpAlIMAWFB/Mv4CJCmIkwqCz86BKkGQZGMqA6ZUT6N01DOsPKKOm lgNcf5iAUVAJ5bPALIRJQMBLkKeZ/4JTMQjADeLYDlOCJMsQAnIfGzCtH2BwgflsNxgE8DCwgBNl PLqFMAkwkFyDFmhwg07ALUQJUrUAiIUMDB8x4pxfAKuFMAmQUSsF4ImmHpYsYAbZwoIURZbDARKk DKxfMSyMBxcPmpgWxkPLDeabD6CSQF+9hyZ8hEGwRIMsC06loERTw/iBQQY1zx6QZ2A8gEg0cAth EgzM3QWYGR/ZIFi2QM/4XBcYmL6zfGB+wHB+A8JC/QJ/cKF+C2ahAzTuYBIMRxDlHGcBTypYN9Qg 1IwPlgUCkAKQhUwTGHgaZWTfAWXmICw8IXjwHkjnVmhI9f843P/lAZIESh5aKBEA1g01CLVoA8uC LG7AKNpqMIu2CcRXPTVEqOFKQCm8F+AtvAmBBcTWhyywbMwjgLt6wlJzzQSBqYjAIMptu5ArYD5M eWcSKnOiVLHfw9vE4KNuEwMAHH/DgYykaMkAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
1,ExprTuple,"14, 3","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAD0AAAAUBAMAAADM7ygkAAAAMFBMVEX////MzMxUVFR2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAACgK8IjAAAAAXRSTlMAQObYZgAAAP9JREFU KM9jYOC7wIAL5IGI0zilGZgeAIl1uOV5FjAw8E7ALc9gzsDA8QCP/FEGhhd4pBm4HBgqgdTxoifF Bag2l+u5Aw1nnsCwF8gx4G3nWIAif5jhRkEX0HELGNqAqg4w/2FBdYYNg/0BPwYGtgaGEKB+Bu4N UPFTq0BgIcMBht0gLuMHkDwDA38BhtO+Q+XbQLS8ALo040cQydIAch/bgXoG9gMo0jnsHxhqwO6z ZGDwT9jP8JaB5xdCmuUb9wdQvHEaMHgwMBwqPz4XyOlAGMGjXlN5FxQ+Akjhy3YAw5EnkOOHGWv8 MOyDcY5jjV9E+sBMR9wgu/Gkr3sMDADjQkH1JDsALwAAAABJRU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" />"
2,Operation,operator: 4operands: 5,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAJcAAAAUBAMAAABylbI8AAAAMFBMVEX////MzMxUVFR2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAACgK8IjAAAAAXRSTlMAQObYZgAAAmhJREFU OMuVVM1rE0EU/213m6zJbly8ekiFguChxFOoCPamJ63gH7BHD2KXegiFigEhtaJlwYsXIRYpnqSI t2JdGkprPSiIaAU1iAc9SGOFFCEH35uZ3SSTD3QOM/tmf/M+fu8DyL3BoJWq6zc/B2JxlbdX/f+Z 108CE9qls9ofXHzrYYQtP+nALkbJ9wq269hRQk6dmXpfXRnfWYJTBdywM67lGO1Ow6zCDBRagXZ7 iHjI+zvgFLkH2F3GrGVPfmTHkGrBmZbS0TvyXNJ0uYvi9VNgJsIm8L37t7E2Jc7DY7CawCN5O16S 0Za7wembkpZ94IKPzBTmSNia/XYtiOk54fPxOoR1AExKLqu2fEzROqWJs0UJrS0oqv+QsgBmiOck FNwFuxqby91n1/cKMEjZR5lZ3xQWslRFNXwIbovbl3fjF7+BfAC3igphI7Nl1ds88L4XwiDMZXFl Iyds2aTyNM5E51kYWUnMk2f5AlJlXCLPkF3VeVBh5sXde+AYn6PEZ4Q1CdxIlKkwjQYrI7pjxhIe KAGpZqyMcnkkVgYcKOiPOExOwIzPyios5T0dQL66LeCcKKZQlSuHCeOXbph9/RLBKnMCUtENpEVw XxPXZdHKBLAiVigScCXdwLxWGrJoKQGTXCMvZAc7z9o1VMEmuXtPlKwKlZNvNbON9mhQRWv7zkXg UIED2ShtPegZHbUScy66ZZw3LlsKxDk+P/dJbyenOEuGM57WTvqUECUvaDf6tVPX2tYavWcOBd3y 7jBl3BfrHY48vsXLSy52NHx2SBg8ggYNR7HW/3E4JoaGjG33f8b2Z+AvDN+kGm194fEAAAAASUVO RK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
3,Operation,operator: 6operand: 14,
4,Literal,,
5,ExprTuple,"8, 9","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAHkAAAAUBAMAAABbvo/+AAAAMFBMVEX////MzMxUVFR2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAACgK8IjAAAAAXRSTlMAQObYZgAAAeFJREFU OMudVLtKA0EUPWuWTUx24/5BtBIsJFZBEUynlQ/wA/YDRIMWQVAUhEQFJaWNEFMEKxGxC+pgkCgW CiLYqEEstBCjQoKSwrn71I1ukSlm7jzOzLnn3jtA+BLNtUnqzv/e8833eCBjVypaytzYcdbkFWbb eZyW/wUHNXkdchZQMj9WpZyFUEbhy/6Bk7aovwb6OAEg8OsFMacaRqgdUr0RrKzox/eAKYYT4On3 tlCI62NbO8RqA9i/ZLj2DoxoCMYxyyel6ceZhHkg3KXRcJGBWDPESHYPxoy9YtrU55OjE/BlcMAn USUdsJ0MbxC51ygEA13ETWJVt87WrCMfQCQBJYsUDw7z1cWy4xr1rxkIH/q8HwNsmIyWvP0AfzsS hbSIcf42Qrtu1xzmDAVj59hGm8yFCqG5RpbXtmtcNclSrWaOzxZzUm1KI3SKZhHVfYCzUcyICW/u q4nNPYO4SKpJbAF+ne+DTc7Jlgl/BXOuiBnZwlXrpdAd4UX3Z98JbQonqvzF86caqjiFZGZLQJPH gNYohrgcydJmQ6EVkx3AMoPcOTd7685UOTbNfQ2qrkx1ZzXzKtFTV5U0lKlngVMGHv6o0O1laqq9 UPICU4X+9zvozfPbCZHPTf9Md8A3C9WGgApCovUAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
6,Literal,,
7,ExprTuple,14,
8,Opera

It is actually common to have only one *condition*, indicating a value that is only defined when the condition is satisfied.

In [75]:
from proveit.logic import Equals, NotEquals
from proveit.numbers import zero, Mult, frac
div_cancel_cond = Conditional(Equals(Mult(x, frac(y, x)), y),
                              NotEquals(y, zero))

div_cancel_cond: {(x * (y / x)) = y if y != 0.

It will then have singular `condition` and `value` attributes.

In [76]:
div_cancel_cond.condition

y != 0

In [77]:
div_cancel_cond.value

(x * (y / x)) = y

<a name="Lambda"></a>Lambda
====

This type of **expression** represents a mathematical mapping or function.  It contains *parameter(s)* (one or more *variables* and/or *iteration* of *indexed* *variables* that will be discussed in the <a href="#IndexedVar">IndexedVar</a> section below) and a *body* (any **expression** that the *parameters* are to be mapped into).

In [78]:
from proveit import Lambda

In [79]:
# maps any value to 5
map_to_5 = Lambda(x, FIVE)

map_to_5: x -> 5

In [80]:
# Let's look at the Expression info
map_to_5.expr_info()

,core type,sub-expressions,expression
0,Lambda,parameter: 1body: 2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAADcAAAAPCAMAAAB3LKe0AAAANlBMVEX///8QEBAoKCgNDQ1mZmaq qqrMzMxEREQyMjIiIiKYmJh2dna6urrc3NxUVFSIiIju7u4AAADPT139AAAAAXRSTlMAQObYZgAA ALtJREFUKM+dUUkSwzAII61XvAD//2xxkrahE88k1QljSVgG4B4SMgTn4S4WUZTbMkilU7tC/GH1 qwO8m+mYqLdKhd6NaJjZ6hDJhVWmKTET5HTO9MYmMkCTqlXRqghDbec6oME1iIMxmtF4Ph/JQFb/ A5zsM4RMIutf81fW1zBlNwoS5j8R0sFmSds8bXkGkhGDz3XpuMBtKeMLq3iIqmv4uUTzSrO/Nk64 qBV3IiaHONm7MxFUWLpj+BsvBH8Gz9YXoekAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
1,Variable,,
2,Literal,,


Here is an example with multiple *parameters* represented by an `ExprTuple` and a `Conditional` *body* .  This maps a pair of real numbers to the first element (invoking the `And`, `Add`, and `InSet` <a href="#Operation">`Operation`</a>s and using the `Real` <a href="#Literal">`Literal`</a>):

In [81]:
# Can have multiple arguments
from proveit.logic import InSet, And
from proveit.numbers import Add, Real
from proveit import Conditional
lambda_expr = Lambda([x, omega], 
                     Conditional(Add(x, omega), 
                                 And(InSet(x, Real), 
                                     InSet(omega, Real))))

lambda_expr: (x, omega) -> {x + omega if x in Real ,  omega in Real.

In [82]:
lambda_expr.expr_info()

,core type,sub-expressions,expression
0,Lambda,parameters: 5body: 1,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAASYAAAAVCAMAAADcg7NfAAAANlBMVEX///8oKCgNDQ3MzMxUVFR2 dna6uroQEBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAADtE0dfAAAAAXRSTlMAQObYZgAA As5JREFUWMPdWOuaoyAMZZf7JQHe/2VHRK0KVKjOfvuVH+1MNeeQQ0gIhPzO8NaTLxuhzyMVuhHR 8W8TiSB0vsi6dfLx7srJvHSCaRgKYiMBonMSPgrmd+YudMdILx03N2WicQ5HKlFEMWTpCLLJEDzg J8RNc2+7Mbh+RCbVC0MsDPsJJPD8xT6RqWluBlKObDzQ4XdkYpdL6MFMwxV+2kB6Yr/bPIzsD9fY AMo+LFOgtEcmChrr4TD5eR2JA+Z6mQlqzYLQx33FNi2zPtzu8A4hWZepClrIdGaZ/oKYMoSV0r4Y SzB02No1bMr+FyRD5ktOR57yvCb7ACR+RcSY5+ZdQxh1VI25NmghU8Eyx+08S7abawWsVnqzn8rY owY1kgFzYlRe/+lXHpGIvelrJXl+S5lW/Oh9ZVCStkELmQqWebqFTBUwqOdgmdKNIpckA+ZEZpmT k3CyEyKXyLQYYanT28O/f9xhxK1wU1irQgHK51eNnL9Yi6UuUzlDKjf2YzgIS071vEYyYL7JlEY8 5Yz5XzqvAydnmcxpHmaX3cUSTTXQIppKlrpMJViAVkW3ggK5Ihkw3zbdrAGtBKpOzwNfxWhsOnrc ygyaoIVMJUtTpjOYbB58AP01Sb/5lsIVEp1CZV8hgaaZ2d1BvZnCTyf5pdLVQAuZSpa6TBUwq1t+ UkYsvSLpN19rvIiKQFy6YIxZDS65lSpaLtcOx76W4tD0iOq56QDalKlkIVsW2uWiKphRjW4jJWpG 37nSZb4WR7fsI2Rao7Y+g5iceHDqAgPRUm5uOdpoKmhNpiNoS6aSJa1mjAzD9Ln1oFUwZCCw0rvS pLCbhG270me+yMnqzYqqt43Ye2J/qlm5HoLDsdvodKXX/G3ry++2vvJta4jk343HLr6gCCfUNy9S bt83PXeZ9ljkltdyvlfPdo/8v9xePnjV3HnJ68MApvquu/AfSnkmoVsvqcEAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
1,Conditional,value: 2condition: 3,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAANMAAAAVBAMAAAAumMZDAAAAMFBMVEX///+IiIi6urrc3NxERETM zMwQEBCqqqpUVFQiIiJ2dnYyMjLu7u5mZmaYmJgAAABN0h6tAAAAAXRSTlMAQObYZgAAAphJREFU SMe9lk1oE0EUx/9Jds12sxu3ICoSsSpKEaRBtGgOskg8iKXZQxAvEoWiKCrxIgiCK+K5fkAlFCF6 aA8qBoofGMENeggiNljsQaFE7aUXSUGpEAvOzG62O/mwUUrmsLzZ/3vzy3vzdiYAwjPo0FBedIqE cGUZh/4y0LumeexIvPp0sHXo3FAiP1R2p/KNZVCpLMSzfT1NtZBiioMDrWNzRSRnrRaoC80iVL3F UkIgC31va5QeIy7mv6CkUt2L2UwmV0NNhRoDajpBqdFc26g9RgMqGfFkNSU0hLg6Q+lelNJ9bfM9 HkWz3kc3KJAwxelzjFVYv2uDRpv2pbeAZq+TOnW+BU4nKMnvFlAi+b3FnDbKocKUMqmxrTchMW8l G77ZRashRT2o4F1n4a/UZR6cTlCXPrttcdEAHuKxtZVDqSzhNFuuhpIt+befriJ42uJ05pljs8TH wel6f2Y07Zb1NnlYiLPJg3x+JJ9/TqwtpFERjnIoBYJd9iN5Mpy2uF9yet3fA9If33hdj+0U3KSw 3aDP+bq20CCWIINDAatZRbHb2+wfkrbpA44BWV7XY+q2pQ9Foqb4sw5Vgi+NWD0qxX4WVnlRwnut VvJ1CGR5nezVwBfutLjsqyDCoQ4iVcIEjwpam+Cj5VA1D0o0P9qmIS4ixEq+pBPUUdXwoPwLQiVQ 5lBjj04tjtmvyAbZe9SXTmA/e3XHczBhXDKUKlnnzPSVT+fB6+RgktPXqTWZYyjlamTjd76Ab6rR tSeYJQ7/ej38g1KL3YUZm148YLnHrSEcXkDGgpKY8J100qnp5Lh9gvgr2oNdZnsHU8PYcSiT886D 1t91p4DHuem7/7qJ5JW5r9oZhfbcVuIWLrfnFuzQf4s//kC5fmap/u0AAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
2,Operation,operator: 4operands: 5,
3,Operation,operator: 6operands: 7,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAJ0AAAAUBAMAAABltyL1AAAAMFBMVEX////MzMxUVFR2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAACgK8IjAAAAAXRSTlMAQObYZgAAAo9JREFU OMuVVE1oE1EQ/jabJtnsZg0FD5KDEbwJJQepiJeoFwXBggpiEfbYkwYFgyCYg8ZYRCJVBMGSgnoo UnMQLDZtloBUIoIHQWtBFn9AEMGTonhw5r3d7EvYBp3DvjfvzX4z876ZAexX2FAS3uDJNwyRU/x5 EXGRubz0+/4ybcYGLqxmFM7byfb8JIUVY/cLURZpK68tt4Cur9vBuRcZWKOD3LoLq0Gh1KMMTLuA 5gdAL/k4vtVadKLNj/RHHtgFpPo8ZhZr0wHebJrym5DnuWtyvd6Ps16rNXw8w6PdM+CLeq89ccP4 Zk3a3JP69rJMu9IHl7sQxmd49LTpIs6Tunrm01mRWNdV8s3vo81uSUMjJdYkpR1j3Evi9KGSbyxO +ep1EItWIXMlJeIeU94vcZXN3wlVd3RHHBOJrxnpJ2sxT8GbukOxZBqokrWr/4mLu2aId7z2gDdT Qk3BFv5ShCqYGRdGIR+fa9PsMFHBUfIGUwJpJ+ZJJN7Oeos3W8XFG2AbryNFxIsgnt6ztpmtfT4O mvxU2nfGAzbJsrCV+Ao3cyEesTvq4yWBLUCBta9KvRh7mxKvKv7KyvMJBc98yU4OiLar+5VM+RrA OdgCb0StvxblgHiF+Ui4F5EUzN5W8LT8jYAPxmJM5sPMar/8NjFKCt6okRV8UEEcdtp+n6ecsN8w HiOLGVFnfs5cD/qRW6dnTkqrqtJvukMvbBQ4oU559a4cMtaOQ715kDWf/gDmRDXzpyzzsdoLyWN+ YJ3HbjAP5rD0yEE6O9Bv8T2y33rzRDREUZAf0W/7F2W/BfJ8o3kQiF7q19cwVGgeYGWYQXdw7njD rHleRc7Tnqz80zzt8zZs3mf+b95T1/wFLZ+wXhD+JzgAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
4,Literal,,
5,ExprTuple,"13, 14","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAC8AAAAUBAMAAADvtHnxAAAAMFBMVEX////MzMxUVFR2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAACgK8IjAAAAAXRSTlMAQObYZgAAAQRJREFU GNNtkTFLw0AYhp+akNQkjVk7+RcyKW4d3RzUUfAXqChUnOw

The `parameters` and `body` attributes access these respective sub-expressions:

In [83]:
lambda_expr.parameters

(x, omega)

In [84]:
lambda_expr.body

{x + omega if x in Real ,  omega in Real.

If a `Lambda` map has one *parameter*, it will have a `parameter` attribute.

In [85]:
single_param_lambda = Lambda(x, Mult(FIVE, a))

single_param_lambda: x -> (5 * a)

In [86]:
single_param_lambda.parameter

x

### Lambda relabeling and relabeled equivalence

The choice of *parameter* *labels* has no effect on the meaning of a `Lambda` map.  We are free to "relabel" the *parameters* (via `Lambda.relabeled`) as long as it is done consistently and distinct labels remain distinct.

In [87]:
lambda_expr

(x, omega) -> {x + omega if x in Real ,  omega in Real.

In [88]:
lambda_expr_v2 = lambda_expr.relabeled({x:y})

lambda_expr_v2: (y, omega) -> {y + omega if y in Real ,  omega in Real.

We can even simultaneously swap labels:

In [89]:
lambda_expr

(x, omega) -> {x + omega if x in Real ,  omega in Real.

In [90]:
lambda_expr.replaced({x:omega, omega:x}, allow_relabeling=True)

(omega, x) -> {omega + x if omega in Real ,  x in Real.

In [91]:
lambda_expr_v3 = lambda_expr.relabeled({x:omega, omega:x})

lambda_expr_v3: (omega, x) -> {omega + x if omega in Real ,  x in Real.

But we cannot allow a collision of the label mappings:

In [92]:
from proveit import DisallowedParameterRelabeling
try:
    lambda_expr.relabeled({x:y, omega:y})
    assert False, ("Expecting a DisallowedParameterRelabeling error; "
                   "should not make it to this point.")
except DisallowedParameterRelabeling as e:
    print("EXPECTED ERROR: ", e)

EXPECTED ERROR:  Cannot relabel omega in (x, omega) -> {x + omega if x in Real ,  omega in Real.; relabeling is only allowed when all occurrences of a range of parameters matches the exact range appearing as parameters (otherwise, the bound verses free portions of the range may be ambiguous).


In [93]:
try:
    lambda_expr.relabeled({x:omega})
    assert False, ("Expecting a DisallowedParameterRelabeling error; "
                   "should not make it to this point.")
except DisallowedParameterRelabeling as e:
    print("EXPECTED ERROR: ", e)

EXPECTED ERROR:  Cannot relabel omega in (x, omega) -> {x + omega if x in Real ,  omega in Real.; relabeling is only allowed when all occurrences of a range of parameters matches the exact range appearing as parameters (otherwise, the bound verses free portions of the range may be ambiguous).


Also, an exception is raised if we attempt to relabel to a non-Variable.

In [94]:
try:
    lambda_expr.relabeled({x:fx})
    assert False, ("Expecting a TypeError error; "
                   "should not make it to this point.")
except TypeError as e:
    print("EXPECTED ERROR: ", e)

EXPECTED ERROR:  May only relabel Variables/IndexedVars to Variables/IndexedVars; may not relabel f(x)


Moreover, **Prove-It** recognizes the different valid relabeled versions of *Lambda* expressions as the *same* **expression** with respect to *meaning*.  They are regarded as equal.

In [95]:
assert lambda_expr == lambda_expr_v2 == lambda_expr_v3

These are recognized as having distinct *styles* of presentation, however.    **Prove-It** provides flexibility with respect to presentation style of expressions while recognizing the *meaning* equivalence when style is the only distinction.  This is the first of many examples to be encountered.

### Lambda scope and substitution rules

It is important to understand that a `Lambda` expression defines a new scope within its *body* with respect to the *parameters*.  This has two important consequences with respect to substitution rules.

The first consequence of the internal `Lambda` scope is that the `Lambda` masks the substitution of variables that happen to be the same as any of the `Lambda` parameters.  For example,

In [96]:
tuple_with_lambda = ExprTuple(x, lambda_expr)

tuple_with_lambda: (x, (x, omega) -> {x + omega if x in Real ,  omega in Real.)

In [97]:
tuple_with_lambda.replaced({x:a_lit}) 

(a, (x, omega) -> {x + omega if x in Real ,  omega in Real.)

Here we see that $x$ is altered external to the `Lambda` expression, but the $x$ within the `Lambda` expression is left alone.  This is fine because the internal $x$ is essentially a different $x$; the scope is different.

For convenience, `substituted` will perform relabeling if the replacement of a `Lambda` *parameter* is another `Variable`.  For example,

In [98]:
tuple_with_lambda.replaced({x:y}) 

(y, (x, omega) -> {x + omega if x in Real ,  omega in Real.)

The inner and outer $y$ *variables* are still regarded as distinct, but it doesn't hurt to relabel the one within the `Lambda` as we wish.  If we do not want this behavior when performing *expression substitution* or *instantiation*, we can *relabel* the map with a distinct variable first.  This feature simply provides a convenient way to perform inner relabeling simultaneously with other substitutions.

The other consequence for of `Lambda` scoping with respect to substitution is that we must not allow any expression within the `Lambda` *body* to be replaced with something that contains one of its *parameters*.  That is because those *parameters* are, with respect to scope, distinct from anything outside of the `Lambda` expression.  So bringing in a replacement defined for external use would have a different meaning when applied internally, and this must not be allowed.  Otherwise, you could alter the meaning of the expression in ways that can invalidate a proof (e.g., via a *instantiation* step which relies upon *expression substitution* internally).  Thus, the parameters of the Lambda function are "reserved" and may not be used in substitution. 

Consider the mapping defined below:

In [99]:
map_to_new_var = Lambda([x, omega], z)

map_to_new_var: (x, omega) -> z

Expressions involving $x$ and/or $w$ are not allowed to be used in the substitutions for $z$. For example, we are not allowed to transform this map to $(x, \omega) \mapsto \omega$ via substitution.  Prove-It will automatically avoid this via automated relabeling.

In [100]:
map_to_new_var.replaced({z:omega})

(x, _a) -> omega

And we are not allowed to change the map to $(x, \omega) \mapsto f(x)$ via substitution.  Again, automatic relabeling avoids this issue.

In [101]:
map_to_new_var.replaced({z:fx})

(_a, omega) -> f(x)

If it does not involve any of the parameter variables, however, that is okay.

In [102]:
tuple_with_lambda.replaced({x:y}) 

(y, (x, omega) -> {x + omega if x in Real ,  omega in Real.)

In [103]:
map_to_new_var.replaced({z:FIVE})

(x, omega) -> 5

### Applying the Lambda map

In addition to the `substituted` method, the `Lambda` class has an `apply` method.  As arguments, the `apply` method should be given the operands to be mapped.  The `apply` method will apply the proper substitution replacements to the `Lambda` *body* to effect the application of the map to the operands.  Internally, <a href="#Operation_sub">*operation substitution*</a> is effected via the `Lambda.apply` method.

In [104]:
lambda_expr

(x, omega) -> {x + omega if x in Real ,  omega in Real.

In [105]:
lambda_expr.apply(fancy_omega_fn, y)

{(omega * omega!) + y if (omega * omega!) in Real ,  y in Real.

### Mocking up **Lambda** operators

What if we wanted to represent an expression involving a `Lambda` function applied to operand(s).  We cannot do this using the `Lambda` and `Operation` expression types directly because an `Operation` is only allowed to have *variable* or *iterated indexed variables* as *operators*.   But it would not be consistent with the **Prove-It** philosophy of *freedom of expression* to completely disallow representing such an expression if so desired.  We can, in fact, represent such a thing in **Prove-It** by making an `Operation` class whose *operands* are the `Lambda` function as well as the operand on which we wish to apply the function. As a demonstration, we define such an `Operation`, called `LambdaApplication`, in <a href="demo_lambda_app_operation.py">demo_lambda_app_operation.py</a>.

In [106]:
# %load demo_lambda_app_operation
'''
Module that defines a LambdaApplication Operation
class for demonstration purposes in this tutorial.
'''

from proveit import Operation, Literal, NamedExprs

class LambdaApplication(Operation):
    _operator_ = Literal('LAMBDA_APPLICATION', r'{\rm LAMBDA\_APPLICATION}')
    
    def __init__(self, lambda_fn, operand):
        Operation.__init__(self, LambdaApplication._operator_, NamedExprs([('lambda_fn',lambda_fn), ('operand',operand)]))
        self.lambda_fn = self.operands['lambda_fn'] # The Lambda function operand
        self.lambda_operand = self.operands['operand'] # The operand of the Lambda function

    @classmethod
    def extract_init_arg_value(operation_class, arg_name, operator, operand):
        '''
        Given a name of one of the arguments of the __init__ method,
        return the corresponding value as determined by the operator and
        operand of the LambdaApplication Operation.
        (This is important so that Prove-It knows how to 'make' an altered
        copy of this Operation).
        '''
        assert isinstance(operand, NamedExprs), "Expecting LambdaApplication operand to be a NamedExprs object"
        if arg_name=='lambda_fn': 
            return operand['lambda_fn']
        elif arg_name=='operand':
            return operand['operand']

    def string(self, **kwargs): # should accept kwargs even when not used (e.g., 'fence')
        return self.lambda_fn.string(fence=True) + '(' + self.lambda_operand.string() + ')'
    
    def latex(self, **kwargs): # should accept kwargs even when not used (e.g., 'fence')
        return self.lambda_fn.latex(fence=True) + '(' + self.lambda_operand.latex() + ')'


In [107]:
from demo_lambda_app_operation import LambdaApplication

In [108]:
lambda_app_expr = LambdaApplication(Lambda(y, Factorial(y)), x)

lambda_app_expr: [y -> y!](x)

Note that the `_operator_` of the `LambdaApplication` is not displayed but is implicit in the theory of the lambda application formatting.  Also, we made use of `NamedExprs` which will be discussed next.  Let us take a look at the expression information.

In [109]:
lambda_app_expr.expr_info(details=True)

0. [y -> y!](x)
   core type: Operation
   class: <class 'demo_lambda_app_operation.LambdaApplication'>
   operator: 1
   operands: 2
1. LAMBDA_APPLICATION
   core type: Literal
   latex_format: {\rm LAMBDA\_APPLICATION}
   theory: tutorial
   class: <class 'proveit._core_.expression.label.literal.Literal'>
   sub-expressions: 
2. {lambda_fn:[y -> y!], operand:x}
   core type: NamedExprs
   class: <class 'proveit._core_.expression.composite.named_exprs.NamedExprs'>
   lambda_fn: 3
   operand: 4
3. y -> y!
   core type: Lambda
   class: <class 'proveit._core_.expression.lambda_expr.lambda_expr.Lambda'>
   parameter: 8
   body: 5
4. x
   core type: Variable
   latex_format: x
   class: <class 'proveit._core_.expression.label.var.Variable'>
   sub-expressions: 
5. y!
   core type: Operation
   class: <class 'demo_operations.Factorial'>
   operator: 6
   operand: 8
6. !
   core type: Literal
   latex_format: !
   theory: tutorial
   class: <class 'proveit._core_.expression.label.literal.Literal'>
   sub-expressions: 
7. (y)
   core type: ExprTuple
   class: <class 'proveit._core_.expression.composite.expr_tuple.ExprTuple'>
   sub-expressions: 8
8. y
   core type: Variable
   latex_format: y
   class: <class 'proveit._core_.expression.label.var.Variable'>
   sub-expressions:

**Axioms** could then be created to define how the `LambdaApplication` **operation** actually operates, but **axioms** will be discussed in a later chapter.

NamedExprs
=========

`NamedExprs` is a composite **expression** that maps string keywords to sub-**expressions**.  The reason that this may be desired is to be explicit in the internal representation about the role of each sub-**expression** so there is no ambiguity between the internal and external representation.  A good example is the `lambda_app_expr` $[y \mapsto y!](x)$ defined above, where one *operand* plays the role of the lambda function and the other *operand* plays the role of the lambda function's operand.

Here is a more rudimentary demonstration of using **NamedExpressions**:

In [110]:
from proveit import NamedExprs
NamedExprs([('left',x), ('right',y), ('product',Multiply(x, y))])

{left:x, right:y, product:(x * y)}

To produce a `NamedExprs` object, you must supply a list of (keyword, `Expression`) pairs.  The keywords must be strings, but can be any string desired.  When the **NamedExprs** is displayed, it will show each of these pairs in the order that they were originally provided.

ExprRange
=========

It is common in mathematical notation to use ellipses ($...$) in representing an unspecified number of parameters or operands.  The `ExprRange` expression type serves this purpose.

To create an `ExprRange`, supply a *parameter*, *body*, and *start* and *end* indices.

In [111]:
from proveit import ExprRange
from proveit import i, j, k
from proveit.numbers import one, Add, frac
it = ExprRange(k, frac(one, Add(x, k)), i, j)

it: (1 / (x + i)), ..(1 / (x + k)).., (1 / (x + j))

For compactness, we only show the start and the end instances.  It is implicit that our index parameter ($k$, which is not explicitly shown) is that our index ranges from $i$ to $j$ in increments of $1$.  (Future style options could be implemented, if desired, to make the representation more explicit at the expense of being less compact without changing any of the inner workings of the `Iter` expression type.)

Looking at the expression information, we see that there are three sub-expression of the `Iter` object: the `lambda_map`, `start_index` and `end_index`.

In [112]:
it.expr_info()

,core type,sub-expressions,expression
0,ExprRange,lambda_map: 1start_index: 2end_index: 3,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAI8AAAAuBAMAAADjDVfrAAAAMFBMVEX////MzMxEREQiIiK6urrc 3NwyMjJmZmaIiIiYmJh2dnZUVFTu7u4QEBCqqqoAAACna3UaAAAAAXRSTlMAQObYZgAAAXpJREFU SMdjYKA+4LtDooa3B7AKn179hyRjeHbtx24QA+cfEl00f9SgUYNoaBAHqQb1YzeIfe+3SpLMqb6f PoFhFIwC6oL/WMEHEtXj0TCMwfkEKMHzDZlBOmC2AREbgIQ1jGF8gGRTNgHxG8w6/QDJtf1OIF6A IcqF02LODaj8F8hVewOG8hPo7mjAZTCyQZyY0hPIMoiYdhCqQSe6Q7oaUAziaW/Aox2hHm7QiSaQ rgWcFswJKAZxsVzA0yxCqIcbVMAL9DXXAa4/LAEoBsWxJ+A2CEk9zCDOAP4HQBsYmBTA3GVpabZp adlA1gF+PF6DqedKS0u+lpa2ASzkD05hCG2wwH6PNwnD1SMCezGYlBdAN0gbb2TB1SMMSgen+34G jgOosfaPC0+cI9TDDeL5ByT8H+gzRKK6iOVDJG6DkNTDDeIDtRpOd5yonIDmtZ146nQk9XCD2A2o lLKZC9BkzpBmEA8kqBgPxAtQpSyNX6BOnUKZux0UZgCtC61W/5pUbwAAAABJRU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" />"
1,Lambda,parameter: 12body: 4,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAF4AAAAsCAMAAADb5v+WAAAANlBMVEX///8oKCgNDQ3MzMxUVFSI iIjc3NyYmJh2dna6urqqqqpmZmYyMjIiIiIQEBBERETu7u4AAAB0zXpNAAAAAXRSTlMAQObYZgAA ATxJREFUWMPtl91uwyAMRtnAgPkJ5v1fdqTT1EASSgpInRTf9KLqkfVhjiljn19moWlsElpEmtm9 uvE3/saPLpiJt2KJzqozZ4TFTxWes1Px0cykY+VgQHbjdSV6Y4dEb5T96zNkX3afOqXoQWJUh0AT ehdlJCBG6qRfCH1Xxi55vt9fLqsY8TljVyt1KyDANu28W/TYFz0yuWmwCIe7vmxU/D1dfYx3nYMv 1tGI/InP3IS9cx9WI4CF4wws/8BH4T+TsIGpEi7xOHY7lnj9bvQEICSChiq+kHA7PV0P5YF5V8OX Em5fPSlTnYJFWcOXEm7vfr2I2xWgH1b1y+NDnEj42uaHV0dbSvhK8chf4HcSbp9ASi++VVJUwe8k 3P5mMSykH0tV634n4fY/mQAEVqlqOHUJj3DOYAOfNfoDNp0Q62z/eG4AAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
2,Variable,,
3,Variable,,
4,Operation,operator: 5operands: 6,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAADEAAAAsBAMAAAA++k1qAAAAMFBMVEX////MzMxEREQiIiK6urrc 3NwyMjJmZmaIiIiYmJh2dnZUVFTu7u4QEBCqqqoAAACna3UaAAAAAXRSTlMAQObYZgAAAMFJREFU OMtjYCAKvD2AXZxn1/4DuPTMHzEy/bhkqu+nT2AYvuA/DvBhkLr30GWcUunoAi9gjH+4ZFj+4JLh MgBTJ7pDuhpQZdgK+KYHMPAs4LRgTkCVOf+g1X8CA9cBrj8sAagy708cOAnMaQxMCmDusrQ027S0 bBCztgCigr8B3QXWTRAheQE0GZ4vrF+AFN+BfgaOAygynB94/nExMPg/0GeIRNXDrcDwEShzuuNE 5QRUGaYHDO09OMINA5xB5gAALgReomfqxbcAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
5,Literal,,
6,ExprTuple,"7, 8","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAFEAAAAUBAMAAAAHNMzaAAAAMFBMVEX////MzMxUVFR2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAACgK8IjAAAAAXRSTlMAQObYZgAAATxJREFU KM9jYOC7wEAY5IGI0/jV8GxtAJJMD4DEOiBmi8GtlGsCSP0CBgZeIOOQ0R9MFelQmj8BRJozMHCA zOXFo9L/AIg8ysDwgoGASguIGxwYKlFUHi96UlyAqnIlQ+48BgbmCQx7kVXyGPC2cyxAUcnzjf3J X6CKBQxtyCqZDzD/YXmAopLtTw0PMMTZGhhCUMxk4N4AZlitWtW6atUiIIszEizA+AFVJTBMCtB8 xKG+HaqyDVWlvACayvMO7x2AFEsDqo/YDtQzsB9AUXmf4XyCAdhHliAuO1DleaCf/RP2M7xFNVOb gV8AqJLTgMED5GrVX3MZOHqA0Vp+fO4FVJXbGNinA23hEoDEJhjkYI8jCDgBSSEQYICq8gwKD5hC GPbBEmIBvlS6ACkl8+JLz9wgNxKVO+4xMAAAXppXOZrwOycAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
7,Literal,,
8,Operation,operator: 9operands: 10,
9,Literal,,


The `lambda_map`, `start_index` and `end_index` attributes provides access to the sub-expression.  The `parameter` and `body` attributes provide convient access to some indirect sub-expressions (the sub-expressions of `lambda_map`).

In [113]:
it.lambda_map

k -> (1 / (x + k))

In [114]:
it.parameter

k

In [115]:
it.body

1 / (x + k)

In [116]:
it.start_index

i

In [117]:
it.end_index

j

We have special formatting features when an `Iter` is used as *operands* of an `Operator`.  As stated previously, **Prove-It** is flexible in its formatting.

In [118]:
add_it = Add(it)

add_it: (1 / (x + i)) + ..(1 / (x + k)).. + (1 / (x + j))

To really understand how **Prove-It** recognizes an expression, one must look at the expression information. 

In [119]:
add_it.expr_info()

,core type,sub-expressions,expression
0,Operation,operator: 11operands: 1,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAKsAAAAuBAMAAAClu/RBAAAAMFBMVEX////MzMxEREQiIiK6urrc 3NwyMjJmZmaIiIiYmJh2dnZUVFTu7u4QEBCqqqoAAACna3UaAAAAAXRSTlMAQObYZgAAAVxJREFU SMdjYKA14LtDC+2nV/+hxFSc2jkpMhan9lFjR40disZyUGYsDu3se79VUmAqhdoHMeBrwCHxYtTY YWfsf6zgA1CGKy0t+Vpa2gYMLcvS0mzT0rJx6wZrHw1boozl+QYjzidQ07XWMIJ5AwXGcitQFghv D2A3dgKUwXMAh8YzeI2dTc2IeMFAk/ili7EnukO6GlAkTlQQbxambgaeDmCM8yzgtGBGTY0LfIk2 FYtuBnaGm8AC5QDXH5YAZAnWB2VEG4upm4FhKsM/oH0MTAqo5RIPww/iXYuhm4HhAAu4xcDfgBbo fKS0QzB0A71gACLlBdAkmDaQYCyGbgYGtgKg0w70M3AcQJE4/2AB0YUGpm6QfgYG/wf6DJGo9vmz TiDWWCy6GRjeA+053XGicgKqBPdM4rtNmLoZGIppk8vMSSqXCAGobhnWjzRotrH+YiqgRXNwB6iZ CwA2ob/noVDrTgAAAABJRU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" />"
1,ExprTuple,2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAKkAAAAyBAMAAADVWuacAAAAMFBMVEX///8yMjLMzMzc3NyYmJgi IiJmZmZERETu7u4QEBCqqqpUVFSIiIi6urp2dnYAAABBmyukAAAAAXRSTlMAQObYZgAAAptJREFU WMO1mM1rE0EUwF+63Y9u0jR7qBcvHr2lf0LFigcPiQdLi5cV6kEUmot4zVFocasICqLNpdBCIQFP Qmlykh6EFDwVhMaboJj041BCsM7uZGZndt2JCy/vsDv78ubHzJs3b94EgEj+ELBkLWz+RoPCdIG1 cg08KtxgjR1JvVNNh7HfSp9mh76t94LS2m+mo679vJC+Gc10JXUp5VgnZSqbeb2KStWpCxYBlWqf BdozXCodpV5BpgaA4iEy1WuRxyNApmb87fUOm5ojCzVxjk3VzilZknZKai5K9Wdvbsiqx5tX5tNA jU8nKxHVU4CpCmBLl8SBi04tXYNiC51KkPUaOpVMnwwXW2Ya0J7Dp1agW0WnThVgdQzUMjTpoC// KcmTvFR00DeGVFQh1DF4QC9Dd24cq9VWxqt1wh5eQWiMoDagpK7c7vkOuu9Pq8EaSyPj1YV6MjVL 0pkdd/t2YodhMeV1oJicTbPkp3xc3UnscIdnl5lYJrTFLLYed3U0ye1GLUjC0gtKanwmkzCKSsLK 6Cmp/yExapMfkdaXj99+ydT1J6qQC+13uWp4WL0hJ3c/aL2AW627MrVzpKAK9oxqwGt6cg94lfGD pMSvElWrPVBQBXtGvQl9XmXAatCswrPgnXec2ZeOQ0LegoGCyuy/O86y41wNVNPUmWaZPI6H22AQ XS37QrlIg9hq5ct0a/n9dRqwE6dRqqm82oT2nJpxw+KHBsF2rgcHMtWrJe8i0Z5TvRoriIh88C9f fbOnzcvUYy15M4v2nEovGLSCDz6svYPPCxEPZBcU4SrYc+pD0Q8ZF2lvLYqJS3sVGUfKM+c5fV3X TqU74RbGsaL9obfBLJu5gXI73qNl7O1QMY6bPBiI/zoEi/IX4RDxKFhw5KQAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
2,ExprRange,lambda_map: 3start_index: 4end_index: 5,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAI8AAAAuBAMAAADjDVfrAAAAMFBMVEX////MzMxEREQiIiK6urrc 3NwyMjJmZmaIiIiYmJh2dnZUVFTu7u4QEBCqqqoAAACna3UaAAAAAXRSTlMAQObYZgAAAXpJREFU SMdjYKA+4LtDooa3B7AKn179hyRjeHbtx24QA+cfEl00f9SgUYNoaBAHqQb1YzeIfe+3SpLMqb6f PoFhFIwC6oL/WMEHEtXj0TCMwfkEKMHzDZlBOmC2AREbgIQ1jGF8gGRTNgHxG8w6/QDJtf1OIF6A IcqF02LODaj8F8hVewOG8hPo7mjAZTCyQZyY0hPIMoiYdhCqQSe6Q7oaUAziaW/Aox2hHm7QiSaQ rgWcFswJKAZxsVzA0yxCqIcbVMAL9DXXAa4/LAEoBsWxJ+A2CEk9zCDOAP4HQBsYmBTA3GVpabZp adlA1gF+PF6DqedKS0u+lpa2ASzkD05hCG2wwH6PNwnD1SMCezGYlBdAN0gbb2TB1SMMSgen+34G jgOosfaPC0+cI9TDDeL5ByT8H+gzRKK6iOVDJG6DkNTDDeIDtRpOd5yonIDmtZ146nQk9XCD2A2o lLKZC9BkzpBmEA8kqBgPxAtQpSyNX6BOnUKZux0UZgCtC61W/5pUbwAAAABJRU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" />"
3,Lambda,parameter: 14body: 6,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAF4AAAAsCAMAAADb5v+WAAAANlBMVEX///8oKCgNDQ3MzMxUVFSI iIjc3NyYmJh2dna6urqqqqpmZmYyMjIiIiIQEBBERETu7u4AAAB0zXpNAAAAAXRSTlMAQObYZgAA ATxJREFUWMPtl91uwyAMRtnAgPkJ5v1fdqTT1EASSgpInRTf9KLqkfVhjiljn19moWlsElpEmtm9 uvE3/saPLpiJt2KJzqozZ4TFTxWes1Px0cykY+VgQHbjdSV6Y4dEb5T96zNkX3afOqXoQWJUh0AT ehdlJCBG6qRfCH1Xxi55vt9fLqsY8TljVyt1KyDANu28W/TYFz0yuWmwCIe7vmxU/D1dfYx3nYMv 1tGI/InP3IS9cx9WI4CF4wws/8BH4T+TsIGpEi7xOHY7lnj9bvQEICSChiq+kHA7PV0P5YF5V8OX Em5fPSlTnYJFWcOXEm7vfr2I2xWgH1b1y+NDnEj42uaHV0dbSvhK8chf4HcSbp9ASi++VVJUwe8k 3P5mMSykH0tV634n4fY/mQAEVqlqOHUJj3DOYAOfNfoDNp0Q62z/eG4AAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
4,Variable,,
5,Variable,,
6,Operation,operator: 7operands: 8,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAADEAAAAsBAMAAAA++k1qAAAAMFBMVEX////MzMxEREQiIiK6urrc 3NwyMjJmZmaIiIiYmJh2dnZUVFTu7u4QEBCqqqoAAACna3UaAAAAAXRSTlMAQObYZgAAAMFJREFU OMtjYCAKvD2AXZxn1/4DuPTMHzEy/bhkqu+nT2AYvuA/DvBhkLr30GWcUunoAi9gjH+4ZFj+4JLh MgBTJ7pDuhpQZdgK+KYHMPAs4LRgTkCVOf+g1X8CA9cBrj8sAagy708cOAnMaQxMCmDusrQ027S0 bBCztgCigr8B3QXWTRAheQE0GZ4vrF+AFN+BfgaOAyg

IndexedVar
=========

To truly understand what can be done with `Iter` expressions, we must see how they can be used with `IndexedVar` expressions.  On its own, an `IndexedVar` if fairly straightforward.  An `IndexedVar` is effectively an `Operation` where the *operator* is the variable being indexed and the *operand* is the *index*, but it does have some special properties related to <a href="#IterParams">iterated parameters</a> which justifies makin `IndexedVar` a core expression type.

In [120]:
from proveit import IndexedVar
a_i = IndexedVar(a, i)

a_i: a_{i}

In [121]:
a_i.expr_info()

,core type,sub-expressions,expression
0,IndexedVar,variable: 1index: 3,
1,Variable,,
2,ExprTuple,3,
3,Variable,,


This "operator" must be a `Variable` or `IndexedVar`:

In [122]:
try:
    IndexedVar(fx, i)
    assert False, ("Expecting a TypeError error; "
                   "should not make it to this point")
except TypeError as e:
    print("EXPECTED ERROR: ", e)

EXPECTED ERROR:  'var' being indexed should be a Variable or IndexedVar itself; got f(x)


And `IndexedVar` may have multiple indices:

In [123]:
a_ij = IndexedVar(a, (i, j))

a_ij: a_{i, j}

The real purpose of the `IndexedVar` expression type is to be used within an iteration.  So there is a convenient method called `var_range` for generating a range of indexed variables.

In [124]:
from proveit import var_range
a_i_to_j = var_range(a, i, j)

a_i_to_j: a_{i}, ..a_{_a}.., a_{j}

### <a name="IterParams"></a>Iterated parameters

When a `Lambda` expression has iterated parameters, a parameter entry that is an `var_range` over `IndexedVar`s, the `apply` method offers some interesting versatility.  Consider the following example representing a map which distributes a product over a sum of an unspecified number of terms.

In [125]:
a_k = IndexedVar(a, k)
distribution_map = Lambda((a_i_to_j, b),
                          Add(ExprRange(k, Mult(a_k, b), i, j)))

distribution_map: (a_{i}, ..a_{_a}.., a_{j}, b) -> ((a_{i} * b) + ..(a_{k} * b).. + (a_{j} * b))

We can apply a map which splits $a_i, ..., a_j$ into multiple entries.  In so doing, the `Iter` of the result is split in the same manner.  For example, consider the following substitution into $a_i, ..., a_j$:

In [126]:
from proveit import m, n
x_1_to_m = var_range(x, one, m)
y_1_to_n = var_range(y, one, n)
a_sub = ExprTuple(x_1_to_m, y_1_to_n, z)

a_sub: (x_{1}, ..x_{_a}.., x_{m}, y_{1}, ..y_{_a}.., y_{n}, z)

But to allow this as a subtitution, we must prove (or assume) that the lengths of these `ExprTuple`s match (that the have the same number of elements).  We could prove this assuming $j-i+1 = m+n+1$, but for this demonstration we will simply assume what we directly require to be true.

In [127]:
from proveit.core_expr_types import Len
a_len = Len(ExprRange(k, k, i, j))
a_required_len = Equals(Len(a_sub), a_len)

a_required_len: |(x_{1}, ..x_{_a}.., x_{m}, y_{1}, ..y_{_a}.., y_{n}, z)| = |(i, ..k.., j)|

Using `a_required_len` as an assumption, we can `apply`  `distribution_map` to the operands $(x_1, ..., x_m, y_1, ..., y_n, z)$ to obtain the following

In [128]:
requirements = []
distribution_map.apply(*a_sub.entries, b, assumptions=[a_required_len],
                      requirements=requirements)

(x_{1} * b) + ..(x_{k} * b).. + (x_{m} * b) + (y_{1} * b) + ..(y_{k} * b).. + (y_{n} * b) + (z * b)

The `requirements` is appended with facts that were needed to make this happen.  Here, there is just one requirement which is the statement we assumed.  It is a **Judgment** proven by *assumption*; these concepts will be discussed in future chapters.

In [129]:
ExprTuple(*requirements)

(|(x_{1}, ..x_{_a}.., x_{m}, y_{1}, ..y_{_a}.., y_{n}, z)| = |(i, ..k.., j)|)

Recall that the `apply` method is used in the implementation of <a href="#Operation_sub">*operation substitution*</a>.  Iterated parameters can be used in that setting as well.  Furthermore, this code is used in *instantiation* of iterated parameters that are universally quantified which will be discussed in a later chapter.

We will remark here that there are limitations in this capability for the sake of simplicity.  We deal with multiple `IndexedVar`s in an iteration, but their subsitutions must be aligned.  For example,

In [130]:
b_k = IndexedVar(b, k)
b_i_to_j = var_range(b, i, j)
dot_prod_map = Lambda((a_i_to_j, b_i_to_j),
                          Add(ExprRange(k, Mult(a_k, b_k), i, j)))

dot_prod_map: (a_{i}, ..a_{_a}.., a_{j}, b_{i}, ..b_{_a}.., b_{j}) -> ((a_{i} * b_{i}) + ..(a_{k} * b_{k}).. + (a_{j} * b_{j}))

In [131]:
requirements = []
dot_prod_map.apply(*a_sub.entries, *a_sub.entries, 
                   assumptions=[a_required_len], requirements=requirements)

(x_{1} * x_{1}) + ..(x_{k} * x_{k}).. + (x_{m} * x_{m}) + (y_{1} * y_{1}) + ..(y_{k} * y_{k}).. + (y_{n} * y_{n}) + (z * z)

As long as the iteration ranges are aligned, we are okay

In [132]:
x_1_to_n = var_range(x, one, n)
y_1_to_m = var_range(y, one, m)
b_sub = ExprTuple(y_1_to_m, x_1_to_n, z)

b_sub: (y_{1}, ..y_{_a}.., y_{m}, x_{1}, ..x_{_a}.., x_{n}, z)

In [133]:
b_required_len = Equals(Len(b_sub), a_len)

b_required_len: |(y_{1}, ..y_{_a}.., y_{m}, x_{1}, ..x_{_a}.., x_{n}, z)| = |(i, ..k.., j)|

In [134]:
requirements = []
dot_prod_map.apply(*a_sub.entries, *b_sub.entries,
                   assumptions=[a_required_len, b_required_len],
                   requirements=requirements)

(x_{1} * y_{1}) + ..(x_{k} * y_{k}).. + (x_{m} * y_{m}) + (y_{1} * x_{1}) + ..(y_{k} * x_{k}).. + (y_{n} * x_{n}) + (z * z)

In [135]:
ExprTuple(*requirements).with_wrapping_at(1)

(|(x_{1}, ..x_{_a}.., x_{m}, y_{1}, ..y_{_a}.., y_{n}, z)| = |(i, ..k.., j)|,  \\ |(y_{1}, ..y_{_a}.., y_{m}, x_{1}, ..x_{_a}.., x_{n}, z)| = |(i, ..k.., j)|)

If the ranges are not aligned, this will fail

In [136]:
b_sub = ExprTuple(y_1_to_n, x_1_to_m, z)

b_sub: (y_{1}, ..y_{_a}.., y_{n}, x_{1}, ..x_{_a}.., x_{m}, z)

In [137]:
b_required_len = Equals(Len(b_sub), a_len)

b_required_len: |(y_{1}, ..y_{_a}.., y_{n}, x_{1}, ..x_{_a}.., x_{m}, z)| = |(i, ..k.., j)|

In [138]:
from proveit import LambdaApplicationError
try:
    dot_prod_map.apply(*a_sub.entries, *b_sub.entries, 
                       assumptions=[a_required_len, b_required_len],
                       requirements=requirements)
    assert False, "Expecting a LambdaApplicationError error; should not make it to this point"
except LambdaApplicationError as e:
    print("Expected LambdaApplicationError: %s"%str(e))

Expected LambdaApplicationError: Failure to apply (a_{i}, ..a_{_a}.., a_{j}, b_{i}, ..b_{_a}.., b_{j}) -> ((a_{i} * b_{i}) + ..(a_{k} * b_{k}).. + (a_{j} * b_{j})) to (x_{1}, ..x_{_a}.., x_{m}, y_{1}, ..y_{_a}.., y_{n}, z, y_{1}, ..y_{_a}.., y_{n}, x_{1}, ..x_{_a}.., x_{m}, z) assuming (|(x_{1}, ..x_{_a}.., x_{m}, y_{1}, ..y_{_a}.., y_{n}, z)| = |(i, ..k.., j)|, |(y_{1}, ..y_{_a}.., y_{n}, x_{1}, ..x_{_a}.., x_{m}, z)| = |(i, ..k.., j)|):
Improper replacement: Improper replacement of (a_{i} * b_{i}), ..(a_{k} * b_{k}).., (a_{j} * b_{j}) via {(a_{i}, ..a_{_a}.., a_{j}): (x_{1}, ..x_{_a}.., x_{m}, y_{1}, ..y_{_a}.., y_{n}, z), (b_{i}, ..b_{_a}.., b_{j}): (y_{1}, ..y_{_a}.., y_{n}, x_{1}, ..x_{_a}.., x_{m}, z), a: {(a_{i}, ..a_{_a}.., a_{j})}, b: {(b_{i}, ..b_{_a}.., b_{j})}}:
When expanding IndexedVar's within an ExprRange whose parameter is the index, their expansion ExprRange indices must all match. (y_{1}, ..y_{_a}.., y_{n}, x_{1}, ..x_{_a}.., x_{m}, z) vs (x_{1}, ..x_{_a}.., x_{m}, y

Such cases may still be handled, but the iterations must first be equated, via axioms/theorems, to split versions withcommon boundaries.  For example, given the following for the $a$ and $b$ substitutions

In [139]:
a_sub

(x_{1}, ..x_{_a}.., x_{m}, y_{1}, ..y_{_a}.., y_{n}, z)

In [140]:
b_sub

(y_{1}, ..y_{_a}.., y_{n}, x_{1}, ..x_{_a}.., x_{m}, z)

If we know that $m$ and $n$ are natural numbers and $m \leq n$, then we can split `a_sub` and `b_sub` respectively into

In [141]:
from proveit.numbers import subtract
a_sub = ExprTuple(x_1_to_m, var_range(y, one, subtract(n, m)),
                  ExprRange(k, IndexedVar(y, Add(subtract(n, m), k)),
                            one, m), z)

a_sub: (x_{1}, ..x_{_a}.., x_{m}, y_{1}, ..y_{_a}.., y_{n - m}, y_{(n - m) + 1}, ..y_{(n - m) + k}.., y_{(n - m) + m}, z)

In [142]:
b_sub = ExprTuple(y_1_to_m, 
                  ExprRange(k, IndexedVar(y, Add(m, k)), 
                            one, subtract(n, m)),
                  x_1_to_m, z)

b_sub: (y_{1}, ..y_{_a}.., y_{m}, y_{m + 1}, ..y_{m + k}.., y_{m + (n - m)}, x_{1}, ..x_{_a}.., x_{m}, z)

Now the `ExpRange` start and end indices line up: $1$ to $m$, $1$ to $n-m$, and $1$ to $m$, and the last entry is a singular element for both of them.  Now we can apply the map:

In [143]:
a_required_len = Equals(Len(a_sub), a_len)
b_required_len = Equals(Len(b_sub), a_len)
dot_prod_map.apply(*a_sub.entries, *b_sub.entries, 
                   assumptions=[a_required_len, b_required_len],
                   requirements=requirements)

(x_{1} * y_{1}) + ..(x_{k} * y_{k}).. + (x_{m} * y_{m}) + (y_{1} * y_{m + 1}) + ..(y_{k} * y_{m + k}).. + (y_{n - m} * y_{m + (n - m)}) + (y_{(n - m) + 1} * x_{1}) + ..(y_{(n - m) + k} * x_{k}).. + (y_{(n - m) + m} * x_{m}) + (z * z)

All of the variables that are indexed by the range parameter (which ranges from the start to the end) must be expanded together unless the indices of the expansion match with iteration indices, not just equal in length.

In [144]:
b_k = IndexedVar(b, k)
b_i_to_j = var_range(b, i, j)
dot_prod_map = Lambda((a_i_to_j),
                      Add(ExprRange(k, Mult(a_k, b_k), i, j)))

dot_prod_map: (a_{i}, ..a_{_a}.., a_{j}) -> ((a_{i} * b_{i}) + ..(a_{k} * b_{k}).. + (a_{j} * b_{j}))

In [145]:
try:
    dot_prod_map.apply(*a_sub.entries, assumptions=[a_required_len],
                       requirements=requirements)
    assert False, "Expecting a LambdaApplicationError error; should not make it to this point"
except LambdaApplicationError as e:
    print("Expected LambdaApplicationError: %s"%str(e))

Expected LambdaApplicationError: Failure to apply (a_{i}, ..a_{_a}.., a_{j}) -> ((a_{i} * b_{i}) + ..(a_{k} * b_{k}).. + (a_{j} * b_{j})) to (x_{1}, ..x_{_a}.., x_{m}, y_{1}, ..y_{_a}.., y_{n - m}, y_{(n - m) + 1}, ..y_{(n - m) + k}.., y_{(n - m) + m}, z) assuming (|(x_{1}, ..x_{_a}.., x_{m}, y_{1}, ..y_{_a}.., y_{n - m}, y_{(n - m) + 1}, ..y_{(n - m) + k}.., y_{(n - m) + m}, z)| = |(i, ..k.., j)|,):
Improper replacement: Improper replacement of (a_{i} * b_{i}), ..(a_{k} * b_{k}).., (a_{j} * b_{j}) via {(a_{i}, ..a_{_a}.., a_{j}): (x_{1}, ..x_{_a}.., x_{m}, y_{1}, ..y_{_a}.., y_{n - m}, y_{(n - m) + 1}, ..y_{(n - m) + k}.., y_{(n - m) + m}, z), a: {(a_{i}, ..a_{_a}.., a_{j})}}:
ExprRange indices failed to match expansion which is necessary because not all of the indexed variables being indexed by the ExprRange parameter are being expanded (a is expanded but b is not): Unable to prove (1, ..k.., m, 1, ..k.., n - m, 1, ..k.., m, m + 1) = (i, ..k.., j) assuming {|(x_{1}, ..x_{_a}.., x_{m}

Let's try an example where the indices do match.  This time, we'll do it properly by proving the equivalences via proofs.  (This is using some advanced concepts to be discussed in later chapters, but we'll probably rearrange this anyway).

In [146]:
from proveit.numbers import subtract, Natural
j_m_1 = subtract(j, one)
j_m_i = subtract(j, i)
j_m_i__in__Natural = InSet(j_m_i, Natural)
a_i_to_j__partition = \
    a_i_to_j.partition(j_m_1, [j_m_i__in__Natural])

a_i_to_j__partition: {(j - i) in Natural} |- (a_{i}, ..a_{_a}.., a_{j}) = (a_{i}, ..a_{_a}.., a_{j - 1}, a_{j})

In [147]:
a_i_to_j__partition.substitution(Lambda(x, Len(x)))

{(j - i) in Natural} |- |(a_{i}, ..a_{_a}.., a_{j})| = |(a_{i}, ..a_{_a}.., a_{j - 1}, a_{j})|

In [148]:
i_to_j = ExprRange(k, k, i, j)

i_to_j: i, ..k.., j

In [149]:
i_to_j__partition = i_to_j.partition(j_m_1, [j_m_i__in__Natural])

i_to_j__partition: {(j - i) in Natural} |- (i, ..a.., j) = (i, ..a.., j - 1, j)

In [150]:
i_to_j__partition.substitution(Lambda(x, Len(x)))

{(j - i) in Natural} |- |(i, ..a.., j)| = |(i, ..a.., j - 1, j)|

In [151]:
a_sub = a_i_to_j__partition.rhs

a_sub: (a_{i}, ..a_{_a}.., a_{j - 1}, a_{j})

Now, with sufficient assumptions, we can explicitly do the expansion for just the $a$ variable, but the $b$ variable will be forced to split at the same locations.

In [152]:
a_required_len = Equals(Len(a_sub), a_len)
assumptions = [a_required_len, InSet(j, Natural), j_m_i__in__Natural]

assumptions: (|(a_{i}, ..a_{_a}.., a_{j - 1}, a_{j})| = |(i, ..k.., j)|, j in Natural, (j - i) in Natural)

In [153]:
requirements = []
dot_prod_map.apply(*a_sub.entries, assumptions=assumptions, requirements=requirements)

(a_{i} * b_{i}) + ..(a_{k} * b_{k}).. + (a_{j - 1} * b_{j - 1}) + (a_{j} * b_{j})

Note that we now have the extra requirement that the indices match, not just the lengths.

In [154]:
ExprTuple(*requirements).with_wrapping_at(1)

(|(a_{i}, ..a_{_a}.., a_{j - 1}, a_{j})| = |(i, ..k.., j)|,  \\ ((j - 1) + 1) = j, (i, ..a.., j - 1, j) = (i, ..a.., j))

If the iteration parameter occurs anywhere other than as an index of an indexed variable, this will also spur the need for the requirement that the indices match and not just the lengths.  For example, consider

In [155]:
dot_prod_and_more_map = Lambda((a_i_to_j, b_i_to_j),
                               Add(ExprRange(k, Mult(a_k, b_k, k),
                                             i, j)))

dot_prod_and_more_map: (a_{i}, ..a_{_a}.., a_{j}, b_{i}, ..b_{_a}.., b_{j}) -> ((a_{i} * b_{i} * i) + ..(a_{k} * b_{k} * k).. + (a_{j} * b_{j} * j))

We cannot handle the following substitution because the indices do not match with the original iteration indices.

In [156]:
a_sub, b_sub = (ExprTuple(x_1_to_m, y_1_to_n, z),
                ExprTuple(y_1_to_m, x_1_to_n, z))

a_sub: (x_{1}, ..x_{_a}.., x_{m}, y_{1}, ..y_{_a}.., y_{n}, z)
b_sub: (y_{1}, ..y_{_a}.., y_{m}, x_{1}, ..x_{_a}.., x_{n}, z)

In [157]:
try:
    a_required_len = Equals(Len(a_sub), a_len)
    b_required_len = Equals(Len(b_sub), a_len)
    dot_prod_and_more_map.apply(*a_sub.entries, *b_sub.entries, 
                                assumptions=[a_required_len,
                                             b_required_len],
                                requirements=requirements)
    assert False, "Expecting a LambdaApplicationError error; should not make it to this point"
except LambdaApplicationError as e:
    print("Expected LambdaApplicationError: %s"%str(e))

Expected LambdaApplicationError: Failure to apply (a_{i}, ..a_{_a}.., a_{j}, b_{i}, ..b_{_a}.., b_{j}) -> ((a_{i} * b_{i} * i) + ..(a_{k} * b_{k} * k).. + (a_{j} * b_{j} * j)) to (x_{1}, ..x_{_a}.., x_{m}, y_{1}, ..y_{_a}.., y_{n}, z, y_{1}, ..y_{_a}.., y_{m}, x_{1}, ..x_{_a}.., x_{n}, z) assuming (|(x_{1}, ..x_{_a}.., x_{m}, y_{1}, ..y_{_a}.., y_{n}, z)| = |(i, ..k.., j)|, |(y_{1}, ..y_{_a}.., y_{m}, x_{1}, ..x_{_a}.., x_{n}, z)| = |(i, ..k.., j)|):
Improper replacement: Improper replacement of (a_{i} * b_{i} * i), ..(a_{k} * b_{k} * k).., (a_{j} * b_{j} * j) via {(a_{i}, ..a_{_a}.., a_{j}): (x_{1}, ..x_{_a}.., x_{m}, y_{1}, ..y_{_a}.., y_{n}, z), (b_{i}, ..b_{_a}.., b_{j}): (y_{1}, ..y_{_a}.., y_{m}, x_{1}, ..x_{_a}.., x_{n}, z), a: {(a_{i}, ..a_{_a}.., a_{j})}, b: {(b_{i}, ..b_{_a}.., b_{j})}}:
ExprRange indices failed to match expansion which is necessary because the ExprRange parameter appears outside of IndexedVar indices: Unable to prove (1, ..k.., m, 1, ..k.., n, n + 1) = (i, .

But if we ensure that indices match up, we can do the expansion as we saw before.  Let's try this for a slightly modified map in which the Lambda parameters only covers the $a$ variables. 

In [158]:
dot_prod_and_more_amap = Lambda((a_i_to_j),
                                Add(ExprRange(k, Mult(a_k, b_k, k), 
                                              i, j)))

dot_prod_and_more_amap: (a_{i}, ..a_{_a}.., a_{j}) -> ((a_{i} * b_{i} * i) + ..(a_{k} * b_{k} * k).. + (a_{j} * b_{j} * j))

In [159]:
a_sub = a_i_to_j__partition.rhs

a_sub: (a_{i}, ..a_{_a}.., a_{j - 1}, a_{j})

In [160]:
a_required_len = Equals(Len(a_sub), a_len)
b_required_len = Equals(Len(b_sub), a_len)
assumptions = [a_required_len, b_required_len, InSet(j, Natural), j_m_i__in__Natural]

assumptions: (|(a_{i}, ..a_{_a}.., a_{j - 1}, a_{j})| = |(i, ..k.., j)|, |(y_{1}, ..y_{_a}.., y_{m}, x_{1}, ..x_{_a}.., x_{n}, z)| = |(i, ..k.., j)|, j in Natural, (j - i) in Natural)

In [161]:
requirements = []
dot_prod_and_more_amap.apply(*a_sub.entries, assumptions=assumptions, requirements=requirements)

(a_{i} * b_{i} * i) + ..(a_{k} * b_{k} * k).. + (a_{j - 1} * b_{j - 1} * (j - 1)) + (a_{j} * b_{j} * j)

Again, we have the extra requirement that the indices match, not just the lengths.

In [162]:
ExprTuple(*requirements).with_wrapping_at(1)

(|(a_{i}, ..a_{_a}.., a_{j - 1}, a_{j})| = |(i, ..k.., j)|,  \\ ((j - 1) + 1) = j, (i, ..a.., j - 1, j) = (i, ..a.., j))

In [163]:
%end basic_expressions

# Next chapter: <a class="ProveItLink" href="tutorial02_proof_basics.ipynb">Proof Basics</a>

## <a class="ProveItLink" href="tutorial00_introduction.ipynb#contents">Table of Contents</a>